<h1>1 - Extracao PDFs - Table Transformer - V0</h1>

## 1. Modules e Config

In [1]:
import os
import csv

import json
from io import StringIO
from pathlib import Path
from PIL import Image

from pdf2image import convert_from_path

import matplotlib.pyplot as plt

import cv2
from PIL import Image
import pytesseract
import torch

from transformers import DetrImageProcessor
from transformers import DetrFeatureExtractor
from transformers import TableTransformerForObjectDetection

model = TableTransformerForObjectDetection.from_pretrained("microsoft/table-transformer-structure-recognition")

Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
import fitz  # Módulo PyMuPDF
import re
from unidecode import unidecode
import PyPDF2


from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTChar

sao joao

In [3]:
#Diretório TESTE raiz para documentos PDF
root_directory = 'novo_modelo/data_pdf'

root_dir_section_images = 'novo_modelo/images/processadas/sections'

target_directory = 'novo_modelo/jsons'


#root_dir_section_images = '.\\.novo_modelo\images\processadas'




src_pdf_nao = 'novo_modelo/data_pdf_nao_pesquisavel'  # Diretório raiz TESTE

tgt_imagens = 'novo_modelo/images'

tgt_pdf_convertido  = 'novo_modelo/data_pdf_convertido'

path_modelo  = "novo_modelo/modelos/modelos.csv"






# Nome do arquivo CSV
csv_filename = "novo_modelo/modelos/modelos.csv"

# path imagem to resize
image_src = "novo_modelo/images/imagem_1.jpg"

# path image resized
img_resi_path = "novo_modelo/images/processadas/im1_resized.jpg"

#Usada para registar os numeros de paginas do PDF
pdf_pages = "data/data_image_test/page_"

# If you don't have tesseract executable in your PATH, include the following:
tessdata_dir_config = '--tessdata-dir "/home/dani-boy/miniconda3/envs/tables-detr/share/tessdata/" --user-patterns "novo_modelo/modelos/user-patterns2.txt" --dpi 600 --oem 3 --psm 3'

# If you don't have tesseract executable in your PATH, include the following:
#pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract'
# Example tesseract_cmd = r'C:\Program Files (x86)\Tesseract-OCR\tesseract'

## 2. Funcoes

### 1. Funcoes de Processamento PDF e Imagem 

In [3]:
def converte2image(doc_path):
    
    #poppler_path = r'C:\poppler-23.08.0\Library\bin'
    #pdf_document = fitz.open(doc_path)
    #page_number = 0
    pages = convert_from_path(doc_path, 500, poppler_path)
    total_pages = len(pages)
    image_counter = 1
    for page in pages:
        image_name = os.path.join(f'{tgt_imagens}/imagem_{str(image_counter)}.jpg')
        page.save(image_name, 'JPEG')
        image_counter = image_counter + 1
      
        
def resizeImage(image_path):
    
    image = Image.open(image_path).convert("RGB")
    newsize = (2067, 2923)
    im1 = image.resize(newsize)
    im1.save(r"./novo_modelo/images/processadas/im1_resized.jpg", 'JPEG')
    print(im1.verify)   
    
   
    
# Leitura do arquivo CSV e criação do dicionário de recortes
def create_cut_dictionary(csv_filename):
    cut_dictionary = {}
    with open(csv_filename, 'r') as csvfile:
        csvreader = csv.DictReader(csvfile)
        for row in csvreader:
            model_name = row['modelo']
            item = row['Item']
            x0 = int(row['x0'])
            y0 = int(row['y0'])
            x1 = int(row['x1'])
            y1 = int(row['y1'])
            
            cut_info = {'Item': item, 'x0': x0, 'y0': y0, 'x1': x1, 'y1': y1}
            
            if model_name not in cut_dictionary:
                cut_dictionary[model_name] = []
            
            cut_dictionary[model_name].append(cut_info)
    
    return cut_dictionary



def cropDocumento(img_resized, section, x0, y0, x1, y1):
    im = Image.open(img_resized)
    left = x0
    top = y0
    right = x1
    bottom = y1
    im_croped_img = im.crop((left, top, right, bottom))
    img_file_path = os.path.join(f"{tgt_imagens}/processadas/sections/{section}.jpg")
    im_croped_img.save(img_file_path, 'JPEG')   



# Efetuar o crop da imagem da NF segmentando por secoes
def cropSections(nf_model, img_resi_path):
    for model_name, cuts in cut_dict.items():
        if model_name == nf_model:
            print(f"Modelo: {model_name}")
            for cut in cuts:
                model_section = cut['Item']
                x0 = cut['x0']
                y0 = cut['y0']
                x1 = cut['x1']
                y1 = cut['y1']
                cropDocumento(img_resi_path, model_section, x0, y0, x1, y1)



# colors for visualization
COLORS = [[0.000, 0.447, 0.741], [0.850, 0.325, 0.098], [0.929, 0.694, 0.125],
          [0.494, 0.184, 0.556], [0.466, 0.674, 0.188], [0.301, 0.745, 0.933]]

def plot_results(pil_img, scores, labels, boxes):
    plt.figure(figsize=(26,20))
    plt.imshow(pil_img)
    ax = plt.gca()
    colors = COLORS * 100
    for score, label, (xmin, ymin, xmax, ymax),c  in zip(scores.tolist(), labels.tolist(), boxes.tolist(), colors):
        ax.add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                   fill=False, color=c, linewidth=3))
        text = f'{model.config.id2label[label]}: {score:0.2f}'
        ax.text(xmin, ymin, text, fontsize=10,
                bbox=dict(facecolor='yellow', alpha=0.3))
    plt.axis('off')
    plt.show()
    
def processaTabela(file_path):
  
  image = Image.open(file_path).convert("RGB")
  width, height = image.size
  image.resize((int(width*0.5), int(height*0.5)))

  feature_extractor = DetrImageProcessor()
  encoding = feature_extractor(image, return_tensors="pt")
  
  with torch.no_grad():
    outputs = model(**encoding)
      
  target_sizes = [image.size[::-1]]
  results = feature_extractor.post_process_object_detection(outputs, threshold=0.6, target_sizes=target_sizes)[0]
  
  return results    


# Classe para representar uma célula da tabela
class CelulaTabela:
    def __init__(self, coordenadas, label, texto):
        self.coordenadas = coordenadas
        self.label = label
        self.texto = texto
        
# Função para recortar a imagem com base nas coordenadas do retângulo
def recortar_imagem(img, coordenadas):
    x_min, y_min, x_max, y_max = coordenadas
    return img[int(y_min):int(y_max), int(x_min):int(x_max)]      


def processaOcrTable(file_path, results):
    dados_da_tabela = {}
    # Carregue a imagem do PDF (ou outra fonte)
    img_pdf = cv2.imread(file_path, cv2.IMREAD_COLOR) 
    
    # Processo de OCR para cada célula e preenchimento do dicionário
    for i in range(results['boxes'].shape[0]):
        retangulo = results['boxes'][i].tolist()
        img_celula = recortar_imagem(img_pdf, retangulo)
        texto_celula = pytesseract.image_to_string(img_celula, config=tessdata_dir_config, lang='por')
        
        # Mapear o ID numérico do rótulo para o rótulo descritivo usando model.config.id2label
        label_id = results['labels'][i].item() if isinstance(results['labels'][i], torch.Tensor) else results['labels'][i]
        label_descritivo = model.config.id2label[label_id]
        
        # Adicionar as informações ao dicionário
        chave_celula = f'celula_{i}'
        
        dados_da_tabela[chave_celula] = {
            'coordenadas': retangulo,
            'label': label_descritivo,
            'texto': texto_celula
        }
    return dados_da_tabela 


cut_dict = create_cut_dictionary(csv_filename)
          

In [ ]:
cut_dict

### 2. Funcoes de Pesquisa diretorio e arquivos

In [11]:
def is_pdf_searchable(pdf_path):
    try:
        pdf_document = fitz.open(pdf_path)
        is_searchable = all(page.get_text("text") != "" for page in pdf_document)
        pdf_document.close()
        return is_searchable
    except Exception as e:
        print(f"Erro ao verificar o PDF: {e}")
        return False

## 3. Funcoes de Extracao de Texto

### 0. Cabecalho

In [12]:
# 1. CABECALHO - DADOS
def extract_fields_dados(text):
    nf_data_dados = {}
    nf_data_dados['secao'] = "1. CABECALHO - DADOS"

    # Extrair Número da Nota
    numero_nota_match = re.search(r'Número da Nota:\s+(\d+)', text)
    if numero_nota_match:
        nr_nro_nf = numero_nota_match.group(1)
        nf_data_dados['numero_nota_fiscal'] = numero_nota_match.group(1)
    else:
        nr_nro_nf = 0    

    # Extrair Competência
    competencia_match = re.search(r'Competência:\s+(.+)', text)
    if competencia_match:
        nf_data_dados['competencia'] = competencia_match.group(1)

    # Extrair Data e Hora de Emissão
    data_emissao_match = re.search(r'ata e Hora da Emissão:\s+(.+)', text)
    if data_emissao_match:
        nf_data_dados['dt_hr_emissao'] = data_emissao_match.group(1)
        
    # Extrair Data e Hora de Emissão
    codigo_verificacao_match = re.search(r'Código Verificação:\s+(.+)', text)
    if codigo_verificacao_match:
        nf_data_dados['codigo_verificacao'] = codigo_verificacao_match.group(1)    
        
    
    return nf_data_dados, nr_nro_nf

# 1. CABECALHO - DADOS
def extract_fields_prefeitura(text):
    nf_data_prefeitura = {}
    nf_data_prefeitura['secao'] = "1. CABECALHO - PREFEITURA"
    # Extrair Nome da Prefeitura
    nome_prefeitura_match = re.search(r'PREFEITURA (.+)', text)
    if nome_prefeitura_match:
        nome_prefeitura = "PREFEITURA " + nome_prefeitura_match.group(1)
        nf_data_prefeitura['nome_prefeitura'] = nome_prefeitura
        
    # Extrair Tipo de NF
    tipo_nf_match = re.search(r'NOTA FISCAL (.+)', text)
    if tipo_nf_match:
        tipo_nf = "NOTA FISCAL " + tipo_nf_match.group(1)
        nf_data_prefeitura['tipo_nota_fiscal'] = tipo_nf 
    
    return  nf_data_prefeitura, nome_prefeitura  


### 1. Prestador de Servico

In [13]:
# 2. PRESTADOR DE SERVIÇO
def extract_fields_prestador_cnpj(text): # Função para extrair campos e valores dentro de um retângulo
    nf_data_prestador_cnpj = {}
    
    nf_data_prestador_cnpj['secao'] = "2. PRESTADOR DE SERVIÇO - CNPJ"
    
    
    # Extrair CPF/CNPJ com máscara 1
    if "CPF/CNPJ:" in text:
        cpf_cnpj_formatado_match = re.search(r'(\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2})', text)
        if cpf_cnpj_formatado_match:
                        nf_data_prestador_cnpj['cpf_cnpj_com_mascara'] = cpf_cnpj_formatado_match.group(1)
                        nf_data_prestador_cnpj['cpf_cnpj_sem_mascara'] = re.sub(r'\D', '', cpf_cnpj_formatado_match.group(1))


    # Extrair Telefone
    telefone_str = None
    
    #telefone_match = re.search(r'Telefone:\s+([0-9.\s-])', text)
    telefone_match = re.search(r'Telefone:\s+([0-9.\s-]+)', text)
    if telefone_match: 
        telefone_str = telefone_match.group(1)
        # Remover quebras de linha
        telefone_str = telefone_str.replace('.', '')
        telefone_str = telefone_str.replace('\n', '')
                
        nf_data_prestador_cnpj['telefone'] = telefone_str
    else:
        nf_data_prestador_cnpj['telefone'] = None   
    
    
    return nf_data_prestador_cnpj 



<mark> 1_frame_prestador_cnpj.jpg - Prestador de servico - CNPJ / Telefone </mark>

In [16]:
for roots, directories, images in os.walk(root_dir_section_images):
                for image in images:
                    #print(f'\n{roots}, {directories}, {images}')
                    #print(image)
                    image_path = os.path.join(roots, image)
                    #print(image_path, image)
                    frame = image
                    
                    
                    if frame == "1_frame_prestador_cnpj.jpg": 
                        image_to_ocr = Image.open(image_path).convert('RGB')
                        text = pytesseract.image_to_string(image_to_ocr, lang='por', config=tessdata_dir_config)
                        nf_data_prestador_cnpj = extract_fields_prestador_cnpj(text)

In [17]:
nf_data_prestador_cnpj

{'secao': '2. PRESTADOR DE SERVIÇO - CNPJ',
 'cpf_cnpj_com_mascara': '33.462.862/0001-95',
 'cpf_cnpj_sem_mascara': '33462862000195',
 'telefone': ''}

<mark> 1_frame_prestador_inscricao.jpg - INSCRICAO MUNICIPAL/ESTADUAL </mark>

In [18]:
for roots, directories, images in os.walk(root_dir_section_images):
                for image in images:
                    #print(f'\n{roots}, {directories}, {images}')
                    #print(image)
                    image_path = os.path.join(roots, image)
                    #print(image_path, image)
                    frame = image
                    
                    # Tomador de serviço
                    
                    
                    if frame == "1_frame_prestador_inscricao.jpg": 
                        image_to_ocr = Image.open(image_path).convert('RGB')
                        text = pytesseract.image_to_string(image_to_ocr, lang='por', config=tessdata_dir_config)
                    

In [19]:
text

'PRESTADOR DE SERVIÇOS\n\nInscrição Municipal:\n4337\n\nInscrição Estadual:\n'

In [20]:
# 1. Prrestador de Servico - INSCRICAO ESTADUAL/MUNICIPAL
nf_data_prestador_incricao = {}
    
nf_data_prestador_incricao['secao'] = "1. PRESTADOR DE SERVIÇO - INSCRICAO"

# Texto de exemplo corrigido
texto = 'PRESTADOR DE SERVIÇOS\nInscrição Municipal:\n4337\nInscrição Estadual:\n'

# Dividir o texto em linhas
linhas = texto.split('\n')

# Inicializar variáveis para armazenar os valores
inscricao_municipal = None
inscricao_estadual = None


# Iterar pelas linhas para identificar os campos e valores
i = 0
while i < len(linhas):
    linha = linhas[i]
    if "Inscrição Municipal:" in linha:
        inscricao_municipal = linhas[i + 1].strip()
        i += 2
    elif "Inscrição Estadual:" in linha: 
        inscricao_estadual = linhas[i + 1].strip()   
        i += 2
    else:
        i += 1
        
if inscricao_municipal == "":
    inscricao_municipal = "None"        
if inscricao_estadual == "":
    inscricao_estadual = "None"

# Imprimir os valores dos campos
print("Inscrição Municipal:", inscricao_municipal)
print("Inscrição Estadual:", inscricao_estadual)

nf_data_prestador_incricao['inscricao_municipal'] = inscricao_municipal
nf_data_prestador_incricao['inscricao_estadual'] = inscricao_estadual






Inscrição Municipal: 4337
Inscrição Estadual: None


In [ ]:
nf_data_prestador_incricao

<mark> 1_frame_prestador_servico.jpg Prestador de servico - Razao social / Endereco / Nome Fantasia / email </mark>

In [21]:
for roots, directories, images in os.walk(root_dir_section_images):
                for image in images:
                    #print(f'\n{roots}, {directories}, {images}')
                    #print(image)
                    image_path = os.path.join(roots, image)
                    #print(image_path, image)
                    frame = image
                    
                    # Tomador de serviço
                    nf_data_prestacao = {}
                    nf_data_prestacao["secao"] = "1. NOVA ROTINA PRESTADOR"
                    if frame == "1_frame_prestador_servico.jpg": 
                        image_to_ocr = Image.open(image_path).convert('RGB')
                        text = pytesseract.image_to_string(image_to_ocr, lang='por', config=tessdata_dir_config)
                        #text = text.replace('\n', '\n\n')
                        #nf_data_tomador = extract_fields_tomador(text) 
                        # Dividir o texto em linhas
                        linhas = text.split('\n')

In [22]:
text

'Nome/Razão Social:\n\nSÃO MARCOS TERRAPLENAGEM E CONSTRUÇÃO LTDA.\nNome de Fantasi\n\nEndereço:\n\nRUA DAS MARGARIDAS ,578 ,SANTA DALILA - Magé-RJ\nE-mail:\n'

In [23]:
# Prestador de servico - Razao social / Endereco / Nome Fantasia / email
text = 'Nome/Razão Social:\n\nSÃO MARCOS TERRAPLENAGEM E CONSTRUÇÃO LTDA.\nNome de Fantasia:\n\nEndereço:\n\nRUA DAS MARGARIDAS ,578 ,SANTA DALILA - Magé-RJ\nE-mail:\n'

nf_data_prestador_servico = {}
nf_data_prestador_servico['secao'] = "2.PRESTADOR DE SERVIÇO - RAZAO"
tipo = None

if "\n\n" in text:
    linhas = text.split('\n\n')
    tipo = 2
    
else:
    linhas = text.split('\n')    
    tipo = 1
   
   
#linhas = text.split('\n')

#linhas = text.split('\n\n')
#print(linhas)


# Inicializar variáveis para armazenar os valores
nome_razao_social = None
nome_fantasia = None
endereco = None
email = None

# Iterar pelas linhas para identificar os campos e valores
i = 0
while i < len(linhas):
    linha = linhas[i]
    #print("linha:", i, linha)
    if "Nome/Razão Social:" in linha:
        if tipo == 2:
            texto1 = linhas[i + 1].strip()
            texto2 = texto1.split('\n')
            nome_razao_social = texto2[i].strip()
        else:
            nome_razao_social = linhas[i + 1].strip()   
        #print("texto_razao: ", texto_razao)
        
        #print()
        #print("texto_razao2: ", texto_razao2)
        #print()
        #print("texto_razao3: ", texto_razao3)
        #print(f'\nRazao Social: {nome_razao_social}')
        i += 2
    elif "Nome de Fantasia:" in linha:
        nome_fantasia = linhas[i + 1].strip()
        i += 2
    elif "Endereço:" in linha:
        
        if tipo == 2:
            texto3 = linhas[i + 1].strip()
            texto4 = texto3.split('\n')
            
            endereco = texto4[0].strip()
            print("endereco:", i, endereco)
            
            
        else:
            endereco = linhas[i + 1].strip()      
        i += 2
    elif "E-mail:" in linha:
        email = linhas[i + 1].strip()
        i += 2
    else:
        i += 1


# Imprimir os valores dos campos
print("Nome/Razão Social:", nome_razao_social)
print("Nome de Fantasia:", nome_fantasia)
print("Endereço:", endereco)
print("E-mail:", email)


nf_data_prestador_servico['razao_social'] = nome_razao_social
nf_data_prestador_servico['nome_fantasia'] = nome_fantasia
nf_data_prestador_servico['endereco'] = endereco
nf_data_prestador_servico['email'] = email

endereco: 2 RUA DAS MARGARIDAS ,578 ,SANTA DALILA - Magé-RJ
Nome/Razão Social: SÃO MARCOS TERRAPLENAGEM E CONSTRUÇÃO LTDA.
Nome de Fantasia: None
Endereço: RUA DAS MARGARIDAS ,578 ,SANTA DALILA - Magé-RJ
E-mail: None


In [ ]:
linhas

In [ ]:
nf_data_prestador_servico

In [ ]:
linhas2

In [ ]:
                        print()
                        print("AQUI", text)
                        
                        #linhas = text.split('\n')      
                        # Inicializar variáveis para armazenar os valores
                        inscricao_municipal = None
                        inscricao_estadual = None
                        
                        # Iterar pelas linhas para identificar os campos e valores
                        i = 0
                        while i < len(linhas):
                            linha = linhas[i]
                            if "Inscrição Municipal:" in linha:
                                inscricao_municipal = linhas[i + 1].strip()
                                i += 2
                            elif "Inscrição Estadual:" in linha: 
                                inscricao_estadual = linhas[i + 1].strip()   
                                i += 2
                            else:
                                i += 1
                        
                    nf_data_prestacao['inscricao_municipal'] = inscricao_municipal
                    nf_data_prestacao['inscricao_estadual'] = inscricao_estadual       
                                         
                     
                    
                    
                    if frame == "1_frame_prestador_servico.jpg": 
                        image_to_ocr = Image.open(image_path).convert('RGB')
                        text = pytesseract.image_to_string(image_to_ocr, lang='por', config=tessdata_dir_config)
                        #text = text.replace('\n', '\n\n')
                        #nf_data_tomador = extract_fields_tomador(text) 
                        # Dividir o texto em linhas
                        print()
                        print(text)
                        linhas = text.split('\n')
                        #print(linhas)

                        # Inicializar variáveis para armazenar os valores
                        nome_razao_social = None
                        nome_fantasia = None
                        endereco = None
                        email = None

                        # Iterar pelas linhas para identificar os campos e valores
                        i = 0
                        while i < len(linhas):
                            linha = linhas[i]
                            if "Nome/Razão Social:" in linha:
                                nome_razao_social = linhas[i + 1].strip()
                                print(f'\nRazao Social: {nome_razao_social}\n')
                                i += 2
                            elif "Nome de Fantasia:" in linha:
                                nome_fantasia = linhas[i + 1].strip()
                                i += 2
                            elif "Endereço:" in linha:
                                endereco = linhas[i + 1].strip()
                                i += 2
                            elif "E-mail:" in linha:
                                email = linhas[i + 1].strip()
                                i += 2
                            else:
                                i += 1


                    nf_data_prestacao['razao_social'] = nome_razao_social
                    nf_data_prestacao['nome_fantasia'] = nome_fantasia
                    nf_data_prestacao['endereco'] = endereco
                    nf_data_prestacao['email'] = email

In [ ]:
nf_data_prestacao

In [ ]:
nf_data_prestador

<h3> Restos - Prestador de servicos </h3>

In [ ]:

# Extrair Inscrição Municipal
    inscricao_municipal_match = re.search(r'Inscrição Municipal:\s+(.+)', text)
    if inscricao_municipal_match:
        nf_data_prestador['inscricao_municipal'] = inscricao_municipal_match.group(1)
        
        
        
        
               
    # Extrair Inscrição Estadual
    #if "Inscrição Estadual:" in text:
    
    # Extrair Inscrição Estadual
    inscricao_estadual_match = re.search(r'Inscrição Estadual:\s+(.+)', text)
    if inscricao_estadual_match:
        inscricao_estadual_str = inscricao_estadual_match.group(1)
        if inscricao_estadual_str == 'Nome/Razão Social:':
            nf_data_prestador['inscricao_estadual'] = "NONE"
        else:    
            nf_data_prestador['inscricao_estadual'] = inscricao_estadual_match.group(1)       
        
                
    
                
    # Nome/Razão Social:
    razao_social_match = re.search(r'Nome/Razão Social:\s+(.+)', text)
    if razao_social_match:
        nf_data_prestador['razao_social'] = razao_social_match.group(1)  
                
    # Nome de Fantasia:
    nome_fantasia_match = re.search(r'Nome de Fantasia:\s+(.+)', text)
    if nome_fantasia_match:
        nf_data_prestador['nome_fantasia'] = nome_fantasia_match.group(1)                                    
                
            
    # Endereço:
    endereco_match = re.search(r'Endereço:\s+(.+)', text)
    if endereco_match:
        nf_data_prestador['endereco'] = endereco_match.group(1) 
    
    # E-mail:
    email_match = re.search(r'E-mail:\s+(.+)', text)
    if email_match:
        nf_data_prestador['email'] = email_match.group(1)  
    else:
        nf_data_prestador['email'] = "NONE"  # Valor padrão quando não há correspondência
   
        

    return nf_data_prestador

###   2. Tomador de Serviço

In [24]:
def extract_fields_tomador_cnpj(text):
    nf_data_tomador_cnpj = {}
    
    
    nf_data_tomador_cnpj['secao'] = "3.TOMADOR DE SERVIÇO"
    
    
    # Extrair CPF/CNPJ com máscara 1
    if "CPF/CNPJ:" in text:
        cpf_cnpj_formatado_match = re.search(r'(\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2})', text)
        if cpf_cnpj_formatado_match:
                        nf_data_tomador_cnpj['cpf_cnpj_com_mascara'] = cpf_cnpj_formatado_match.group(1)
                        nf_data_tomador_cnpj['cpf_cnpj_sem_mascara'] = re.sub(r'\D', '', cpf_cnpj_formatado_match.group(1))

    
    # Extrair Telefone
    telefone_match = re.search(r'Telefone:\s+(.+)', text)
    if telefone_match:
        telefone_str = telefone_match.group(1)
        if telefone_str == 'Inscrição Estadual:':
            nf_data_tomador_cnpj['telefone'] = "NONE"  # Valor padrão quando não há correspondência
        elif telefone_str == '':
            nf_data_tomador_cnpj['telefone'] = "NONE"  # Valor padrão quando não há correspondência
                    
        else:    
            nf_data_tomador_cnpj['telefone'] = telefone_match.group(1)
            
    
    
    # Extrair Inscrição Municipal
    inscricao_municipal_match = re.search(r'Inscrição Municipal:\s+(.+)', text)
    if inscricao_municipal_match:
        inscricao_municipal_str = inscricao_municipal_match.group(1)
        if inscricao_municipal_str == "Telefone:": 
            nf_data_tomador_cnpj['inscricao_municipal'] = "NONE"
        else:    
            nf_data_tomador_cnpj['inscricao_municipal'] = inscricao_municipal_str
    
    insc_municipal_match = re.search(r'INSC:MUNICIPAL:\s+(.+)', text)
    if insc_municipal_match:
        insc_municipal_str = insc_municipal_match.group(1)
        if insc_municipal_str == "Telefone:":
            nf_data_tomador_cnpj['inscricao_municipal'] = "NONE" 
        else:    
            nf_data_tomador_cnpj['inscricao_municipal'] = insc_municipal_str
    else:
        nf_data_tomador_cnpj['inscricao_municipal'] = "NONE"
                        
            
            
    
    return nf_data_tomador_cnpj

def extract_fields_tomador_inscricao(text):
    nf_data_tomador_inscricao = {}
    
    # Extrair RG    
    rg_match = re.search(r'RG:\s+(.+)', text)   
    if rg_match:
        rg_str = rg_match.group(1)
        if rg_str == 'Inscrição Estadual:':
            nf_data_tomador_inscricao['rg'] = "NONE"  # Valor padrão quando não há correspondência
        else:    
            nf_data_tomador_inscricao['rg'] = rg_match.group(1)  
 
        
                
    # Extrair Inscrição Estadual
    inscricao_estadual_match = re.search(r'Inscrição Estadual:\s+(.+)', text)
    if inscricao_estadual_match:
        inscricao_estadual_str = inscricao_estadual_match.group(1)
        if inscricao_estadual_str == '':
            nf_data_tomador_inscricao['inscricao_estadual'] = "NONE"
        else:    
            nf_data_tomador_inscricao['inscricao_estadual'] = inscricao_estadual_match.group(1)   
    else: 
        nf_data_tomador_inscricao['inscricao_estadual'] = "NONE"           

    return nf_data_tomador_inscricao
                                                     
            

In [ ]:
for roots, directories, images in os.walk(root_dir_section_images):
                for image in images:
                    #print(f'\n{roots}, {directories}, {images}')
                    #print(image)
                    image_path = os.path.join(roots, image)
                    #print(image_path, image)
                    frame = image
                    
                    
                    nf_data_tomador = {}
                    nf_data_tomador['secao'] = "2. PRESTADOR DE SERVIÇO"
                    # Tomador de serviço
                    if frame == "2_frame_tomador_cnpj.jpg": 
                        image_to_ocr = Image.open(image_path).convert('RGB')
                        text = pytesseract.image_to_string(image_to_ocr, lang='por', config=tessdata_dir_config)
                        nf_data_tomador_cnpj = extract_fields_tomador_cnpj(text)
                        
                    if frame == "2_frame_tomador_inscricao.jpg": 
                        image_to_ocr = Image.open(image_path).convert('RGB')
                        text = pytesseract.image_to_string(image_to_ocr, lang='por', config=tessdata_dir_config)
                        nf_data_tomador_inscricao = extract_fields_tomador_inscricao(text)       
                    
                    
                    if frame == "2_frame_tomador_servico.jpg": 
                        image_to_ocr = Image.open(image_path).convert('RGB')
                        text = pytesseract.image_to_string(image_to_ocr, lang='por', config=tessdata_dir_config)

                        print()
                        #print(text)
                        
                        # Dividir o texto em linhas
                        linhas = text.split('\n')
                        #print(linhas)
                        # Inicializar variáveis para armazenar os valores
                        nome_razao_social = None
                        endereco = None
                        email = None

                        # Iterar pelas linhas para identificar os campos e valores
                        i = 0
                        while i < len(linhas):
                            linha = linhas[i]
                            if "Nome/Razão Social:" in linha:
                                nome_razao_social = linhas[i + 1].strip()
                                
                                i += 2
                            elif "Endereço:" in linha:
                                endereco = linhas[i + 1].strip()
                                
                                i += 2
                            elif "E-mail:" in linha:
                                email = linhas[i + 1].strip()
                                
                                i += 2
                            else:
                                i += 1

                    #print(f'\nRazao Social: {nome_razao_social}\n')
                    #print(f'\nEndereco: {endereco}\n')
                    #print(f'\nEmail: {email}\n')
                        
                    nf_data_tomador['razao_social'] = nome_razao_social
                    nf_data_tomador['endereco'] = endereco
                    nf_data_tomador['email'] = email

In [26]:
nf_data_tomador

{'secao': '2. PRESTADOR DE SERVIÇO',
 'razao_social': 'MUNICÍPIO DE MACAÉ',
 'endereco': 'PRESIDENTE FEL É Nº 534 PAÇ: BAIRRO: CENTRO CIDADE: MACAÉ - RJ CEP: 27913080',
 'email': ''}

In [27]:
# Texto de exemplo corrigido
texto = 'Nome/Razão Social:\nIKINHA INDUSTRIA TEXTIL LTDA\nEndereço: o o . o\nPROFESSOR STROELLER Nº 953 - ATE 953 - LADO ÍMPAR BAIRRO: QUARTEIRÃO BRASILEIRO CIDADE: PETRÓPOLIS - RJ CEP: 25680175\nE-mail:\nNÃO INFORMADO'

# Dividir o texto em linhas
linhas = texto.split('\n')

# Inicializar variáveis para armazenar os valores
nome_razao_social = None
endereco = None
email = None

# Iterar pelas linhas para identificar os campos e valores
i = 0
while i < len(linhas):
    linha = linhas[i]
    if "Nome/Razão Social:" in linha:
        nome_razao_social = linhas[i + 1].strip()
        i += 2
    elif "Endereço:" in linha:
        endereco = linhas[i + 1].strip()
        i += 2
    elif "E-mail:" in linha:
        email = linhas[i + 1].strip()
        i += 2
    else:
        i += 1


nf_data_tomador['razao_social'] = nome_razao_social
nf_data_tomador['endereco'] = endereco
nf_data_tomador['email'] = email


In [28]:
# Texto de exemplo
texto = "Nome/Razão Social:\nMUNICÍPIO DE MACAE\nEndereço: ê à\nPRESIDENTE FELICIANO SODRE Nº 534 PAÇO MUNICIPAL BAIRRO: CENTRO CIDADE: MACAE - RJ CEP: 27913080\nE-mail:\nNÃO INFORMADO\n"

# Dividir o texto em linhas
linhas = texto.split('\n')

# Inicializar variáveis para armazenar os valores
nome_razao_social = None
endereco = None
email = None

# Iterar pelas linhas para identificar os campos e valores
i = 0
while i < len(linhas):
    linha = linhas[i]
    if "Nome/Razão Social:" in linha:
        nf_data_tomador['razao_social'] = linhas[i + 1].strip()
        i += 2
    elif "Endereço:" in linha:
        nf_data_tomador['email'] = linhas[i + 1].strip()
        i += 2
    elif "E-mail:" in linha:
        nf_data_tomador['email'] = linhas[i + 1].strip()
        i += 2
    else:
        i += 1

# Imprimir os valores dos campos
print("Nome/Razão Social:", nome_razao_social)
print("Endereço:", endereco)
print("E-mail:", email)








Nome/Razão Social: None
Endereço: None
E-mail: None


In [29]:
text

'CPF/CNPJ:\n\n29.115.474/0001-60 |\nINSC:MUNICIPAL:\nTelefone:\n'

In [30]:
nf_data_tomador

{'secao': '2. PRESTADOR DE SERVIÇO',
 'razao_social': 'MUNICÍPIO DE MACAE',
 'endereco': 'PRESIDENTE FELICIANO SODRE Nº 534 PAÇO MUNICIPAL BAIRRO: CENTRO CIDADE: MACAE - RJ CEP: 27913080',
 'email': 'NÃO INFORMADO'}

In [31]:

nf_data_tomador_cnpj


{'secao': '3.TOMADOR DE SERVIÇO',
 'cpf_cnpj_com_mascara': '29.115.474/0001-60',
 'cpf_cnpj_sem_mascara': '29115474000160',
 'inscricao_municipal': 'NONE'}

In [ ]:
nf_data_tomador_inscricao

### 3 - Descricao dos Servicos - 3_frame_descricao_servico.jpg

In [32]:
for roots, directories, images in os.walk(root_dir_section_images):
                for image in images:
                    #print(f'\n{roots}, {directories}, {images}')
                    image_path = os.path.join(roots, image)
                    #print(image_path, image)
                    frame = image
                    
                    if frame == "3_frame_descricao_servico.jpg": 
                        image_to_ocr = Image.open(image_path).convert('RGB')
                        text = pytesseract.image_to_string(image_to_ocr, lang='por', config=tessdata_dir_config)
                        #nf_data_prestador = extract_fields_prestador(text)  

In [33]:
text

'DISCRIMINAÇÃO DOS SERVIÇOS\n\nSer Tapa Buraco), na cidade de Macaé/RJ, abrangendo Perímetro Urb:\nco EDRna Contraio dista - pm Bias e 502/2023. M. Obra (10%)= R$ 104.468,45 Retenção p/ EM Rae E 1 AM, il E ITAU Fenda\n0726 C/Corrente: 35410\n\n'

In [34]:
nf_data_servico = {}
nf_data_servico['secao'] = "4. DESCRIMINACAO DOS SERVIÇOS"

text = text.replace('\n', ' ')
label = "DISCRIMINAÇÃO DOS SERVIÇOS"
if text.startswith(label):
    text = text[len(label):].strip()
nf_data_servico['discriminacao_servicos'] = text

In [35]:
nf_data_servico

{'secao': '4. DESCRIMINACAO DOS SERVIÇOS',
 'discriminacao_servicos': 'Ser Tapa Buraco), na cidade de Macaé/RJ, abrangendo Perímetro Urb: co EDRna Contraio dista - pm Bias e 502/2023. M. Obra (10%)= R$ 104.468,45 Retenção p/ EM Rae E 1 AM, il E ITAU Fenda 0726 C/Corrente: 35410'}

### 5. Valor Total : 4_frame_valor_total.jpg

In [36]:
for roots, directories, images in os.walk(root_dir_section_images):
                for image in images:
                    #print(f'\n{roots}, {directories}, {images}')
                    image_path = os.path.join(roots, image)
                    #print(image_path, image)
                    frame = image
                    
                    if frame == "4_frame_valor_total.jpg": 
                        image_to_ocr = Image.open(image_path).convert('RGB')
                        text = pytesseract.image_to_string(image_to_ocr, lang='por', config=tessdata_dir_config)
                        #nf_data_prestador = extract_fields_prestador(text)    

In [37]:
text

'VALOR TOTAL DA NOTA: R$ 1.044.684,55\n'

In [38]:
nf_data_valor_total = {}
nf_data_valor_total['secao'] = "5. VALOR TOTAL"

valor_total_match = re.search(r'R\$ ([\d,.]+)', text)
if valor_total_match:
    valor_total_sem_formatacao = valor_total_match.group(1).replace('.', '').replace(',', '.')
    nf_data_valor_total['valor_total_nota'] = float(valor_total_sem_formatacao)
                    

In [39]:
nf_data_valor_total

{'secao': '5. VALOR TOTAL', 'valor_total_nota': 1044684.55}

### 6. CNAE e Item da Lista de Serviços : 5_frame_cnae_itens.jpg

In [40]:
for roots, directories, images in os.walk(root_dir_section_images):
                for image in images:
                    #print(f'\n{roots}, {directories}, {images}')
                    image_path = os.path.join(roots, image)
                    #print(image_path, image)
                    frame = image
                    
                    if frame == "5_frame_cnae_itens.jpg": 
                        image_to_ocr = Image.open(image_path).convert('RGB')
                        text = pytesseract.image_to_string(image_to_ocr, lang='por', config=tessdata_dir_config)
                        #nf_data_prestador = extract_fields_prestador(text)   

In [41]:
text

'CNAE - 4313400 - OBRAS DE o\nItem dE Lista de Serviços - 7.02 - CUÇÃO, “POR ADMINISTRAÇÃO, EMPREITADA OU SUBEMPREITADA, DE OBRAS DE Rs CIVIL, HIDRÁULICA\nOU ELÉTRICA E DE OUTRAS OBRAS SEMELHANTES, INCLUSIVE SONDAGEM, PERFURAÇÃO DE POÇOS, ESCAVAÇÃO, DRENAG\n'

In [42]:
# Extrair CNAE
nf_data_CNAE = {}
nf_data_CNAE['Secao'] = "6. CNAE e Item da Lista de Serviços"

linhas = text.split('º\n')

texto_cnae = linhas[0].replace(' . .', '')
cnae = re.sub(r'^CNAE - ', '', texto_cnae, count=1)


# Item da Lista de Serviços 
texto_novo = linhas[1]
texto_novo = texto_novo.replace('\n', '')

item_lista_servico = re.sub(r'^Item da Lista de Serviços - ', '', texto_novo, count=1)


nf_data_CNAE['cnae'] = cnae
nf_data_CNAE['item_lista_servicos'] = item_lista_servico

IndexError: list index out of range

In [ ]:
nf_data_CNAE

### 7. VALORES E IMPOSTOS - 6_frame_valores_impostos.jpg

## IMPORTANTE: Teste Tesseract

### Python Tesseract 


https://github.com/madmaze/pytesseract/tree/master

In [ ]:
from PIL import Image

import pytesseract

In [55]:
image_path = "novo_modelo/images/processadas/sections/im1_resized_item_6.jpg"

In [34]:
image_path = "novo_modelo/images/processadas/sections/6_frame_valores_impostos.jpg"

In [23]:
image_path = "novo_modelo/images/processadas/sections/Captura de tela 2023-08-22 145943.png"

In [31]:
image_path = "novo_modelo/images/processadas/sections/Captura de tela 2023-08-22 145853.png"

In [24]:
print(pytesseract.image_to_string(Image.open(image_path)))

VALOR PIS: VALOR COFINS: VALOR IR: VALOR INSS: | VALORCSLL: OUTRAS RETENGOES: VALOR LiQUIDO:
R$ 0,00 R$ 0,00 R$ 0,00 R$ 11.491,53 R$ 0,00 R$ 0,00 R$ 1.012.299,33



In [35]:
texto = pytesseract.image_to_string(Image.open(image_path))

In [36]:
texto

'VALOR SERVICOS: VALOR DESC. INCOND: BASE DE ALIQUOTA: VALOR ISS: VALOR ISS DESC. COND:\n\nR$ 1.044.684,55 DEDUGAO: R$ 0,00 CALCULO: 2% R$ 0,00 RETIDO: R$ 0,00\nR$ 0,00 R$ 1.044.684,55 R$ 20.893,69\nVALOR PIS: VALOR COFINS: VALOR IR: VALOR INSS: VALOR CSLL: OUTRAS RETENCOES: VALOR LIQUIDO:\n\nR$ 0,00 R$ 0,00 R$ 0,00 R$ 11.491,53 R$ 0,00 R$ 0,00 R$ 1.012.299,33\n'

In [ ]:
# In order to bypass the image conversions of pytesseract, just use relative or absolute image path
# NOTE: In this case you should provide tesseract supported images or tesseract will return error
print(pytesseract.image_to_string(image_path))

In [ ]:
# List of available languages
print(pytesseract.get_languages(config=''))

In [ ]:
# Get bounding box estimates
print(pytesseract.image_to_boxes(Image.open(image_path)))

In [ ]:
# Get verbose data including boxes, confidences, line and page numbers
print(pytesseract.image_to_data(Image.open(image_path)))

In [ ]:
# Get bounding box estimates
print(pytesseract.image_to_boxes(Image.open('test.png')))

# Get verbose data including boxes, confidences, line and page numbers
print(pytesseract.image_to_data(Image.open('test.png')))

# Get information about orientation and script detection
print(pytesseract.image_to_osd(Image.open('test.png')))

# Get a searchable PDF
pdf = pytesseract.image_to_pdf_or_hocr('test.png', extension='pdf')
with open('test.pdf', 'w+b') as f:
    f.write(pdf) # pdf type is bytes by default

# Get HOCR output
hocr = pytesseract.image_to_pdf_or_hocr('test.png', extension='hocr')

# Get ALTO XML output
xml = pytesseract.image_to_alto_xml('test.png')

In [ ]:
# Get HOCR output
hocr = pytesseract.image_to_pdf_or_hocr(image_path, extension='hocr')

In [ ]:
with open('test_modelo.hocr', 'w+b') as f:
    f.write(hocr) # pdf type is bytes by default

In [ ]:
hocr

In [ ]:
# Get ALTO XML output
xml = pytesseract.image_to_alto_xml(image_path)

In [ ]:
xml

In [ ]:
with open('test.xml', 'w+b') as f:
    f.write(xml) # pdf type is bytes by default

In [ ]:
import cv2

img_cv = cv2.imread(image_path)

# By default OpenCV stores images in BGR format and since pytesseract assumes RGB format,
# we need to convert from BGR to RGB format/mode:
img_rgb = cv2.cvtColor(img_cv, cv2.COLOR_BGR2RGB)
print(pytesseract.image_to_string(img_rgb))



In [ ]:
img_rgb = Image.frombytes('RGB', img_cv.shape[:2], img_cv, 'raw', 'BGR', 0, 0)
print(pytesseract.image_to_string(img_rgb))

In [ ]:
# Example of adding any additional options
custom_oem_psm_config = r'--oem 3 --psm 6'
pytesseract.image_to_string(image_path, config=custom_oem_psm_config)

In [37]:


# Example of using pre-defined tesseract config file with options
cfg_filename = 'words'
pytesseract.run_and_get_output(image_path, extension='txt', config=cfg_filename)

'VALOR SERVICOS: VALOR DESC. INCOND: BASE DE ALIQUOTA: VALOR ISS: VALOR ISS DESC. COND:\n\nR$ 1.044.684,55 DEDUGAO: R$ 0,00 CALCULO: 2% R$ 0,00 RETIDO: R$ 0,00\nR$ 0,00 R$ 1.044.684,55 R$ 20.893,69\nVALOR PIS: VALOR COFINS: VALOR IR: VALOR INSS: VALOR CSLL: OUTRAS RETENCOES: VALOR LIQUIDO:\n\nR$ 0,00 R$ 0,00 R$ 0,00 R$ 11.491,53 R$ 0,00 R$ 0,00 R$ 1.012.299,33\n'

In [51]:
# If you don't have tesseract executable in your PATH, include the following:
tessdata_dir_config_novo = '--tessdata-dir "/home/dani-boy/miniconda3/envs/tables-detr/share/tessdata/" --user-patterns "novo_modelo/modelos/user-patterns2.txt" --dpi 600 --oem 3 --psm 3'

In [56]:
textos = pytesseract.run_and_get_output(image_path, extension='txt', lang='por', config=tessdata_dir_config_novo)

In [82]:
texto = textos

In [79]:
original_string

'VALOR SERVIÇOS:  VALO RE po BASE DE ALÍQUOTA: VALOR ISS: VALOR ISS DESC. COND:\n\nR$ 1.044.684,55 DEDUGÃO: R$0 CÁLCULO: 2% R$ 0,00 RETIDO: R$ 0,00\nR$ 0,00 R$ 1.044.684,55 R$ 20.893,69\nVALOR PIS: VALOR COFINS: VALOR IR: VALOR INSS: VALOR CSLL: OUTRAS RETENÇÕES: VALOR LÍQUIDO:\n\nR$ 0,00 R$ 0,00 R$ 0,00 R$11491,53  R$0,00 R$ 0,00 R$ 1.012.299,33\n'

In [83]:
import re

#texto = 'VALOR SERVIÇOS:  VALO RE po BASE DE ALÍQUOTA: VALOR ISS: VALOR ISS DESC. COND:\n\nR$ 1.044.684,55 DEDUGÃO: R$0 CÁLCULO: 2% R$ 0,00 RETIDO: R$ 0,00\nR$ 0,00 R$ 1.044.684,55 R$ 20.893,69\nVALOR PIS: VALOR COFINS: VALOR IR: VALOR INSS: VALOR CSLL: OUTRAS RETENÇÕES: VALOR LÍQUIDO:\n\nR$ 0,00 R$ 0,00 R$ 0,00 R$11491,53  R$0,00 R$ 0,00 R$ 1.012.299,33\n'

# Encontrar os valores monetários no texto
valores_monetarios = re.findall(r'R\$ ([\d.,]+)', texto)

# Encontrar os percentuais no texto
percentuais = re.findall(r'(\d+)%', texto)

# Criar uma lista de valores formatados
valores_formatados = []

# Formatar os valores monetários
for valor in valores_monetarios:
    valor_formatado = valor.replace('.', '').replace(',', '.')
    valores_formatados.append(valor_formatado)

# Adicionar os percentuais formatados
valores_formatados.extend(percentuais)

# Transformar todos os valores em uma única string
resultado = ', '.join(valores_formatados)

print(resultado)








1044684.55, 0.00, 0.00, 0.00, 1044684.55, 20893.69, 0.00, 0.00, 0.00, 0.00, 1012299.33, 2


In [59]:
import re

# String fornecida
texto = 'VALOR SERVIÇOS:  VALO RE po BASE DE ALÍQUOTA: VALOR ISS: VALOR ISS DESC. COND:\n\nR$ 1.044.684,55 DEDUGÃO: R$0 CÁLCULO: 2% R$ 0,00 RETIDO: R$ 0,00\nR$ 0,00 R$ 1.044.684,55 R$ 20.893,69\nVALOR PIS: VALOR COFINS: VALOR IR: VALOR INSS: VALOR CSLL: OUTRAS RETENÇÕES: VALOR LÍQUIDO:\n\nR$ 0,00 R$ 0,00 R$ 0,00 R$11491,53  R$0,00 R$ 0,00 R$ 1.012.299,33\n'

# Lista de chaves esperadas
chaves_esperadas = [
    'VALOR SERVIÇOS', 'BASE DE', 'ALÍQUOTA', 'VALOR ISS', 'DESC. COND',
    'DEDUGÃO', 'CÁLCULO', 'RETIDO', 'VALOR PIS', 'VALOR COFINS',
    'VALOR IR', 'VALOR INSS', 'VALOR CSLL', 'OUTRAS RETENÇÕES', 'VALOR LÍQUIDO'
]

# Padrão de regex para valores monetários e percentuais
padrao_monetario = r'R\$ ([\d,.]+)'
padrao_percentual = r'([\d.]+)%'

# Função para extrair valores monetários e percentuais
def extrair_valores(texto, chave):
    padrao_chave = re.escape(chave)
    padrao_completo = f'{padrao_chave}.*?({padrao_monetario}|{padrao_percentual})'
    valor_encontrado = re.search(padrao_completo, texto, re.DOTALL)
    if valor_encontrado:
        valor_formatado = valor_encontrado.group(1).replace('.', '').replace(',', '.')
        return valor_formatado
    return 'Valor não encontrado'

# Criar lista de valores
valores = []
for chave in chaves_esperadas:
    valor = extrair_valores(texto, chave)
    valores.append(valor)

# Imprimir lista de valores
for chave, valor in zip(chaves_esperadas, valores):
    print(f"{chave}: {valor}")


VALOR SERVIÇOS: R$ 1044684.55
BASE DE: R$ 1044684.55
ALÍQUOTA: R$ 1044684.55
VALOR ISS: R$ 1044684.55
DESC. COND: R$ 1044684.55
DEDUGÃO: 2%
CÁLCULO: 2%
RETIDO: R$ 0.00
VALOR PIS: R$ 0.00
VALOR COFINS: R$ 0.00
VALOR IR: R$ 0.00
VALOR INSS: R$ 0.00
VALOR CSLL: R$ 0.00
OUTRAS RETENÇÕES: R$ 0.00
VALOR LÍQUIDO: R$ 0.00


In [58]:
print(textos)

VALOR SERVIÇOS:  VALO RE po BASE DE ALÍQUOTA: VALOR ISS: VALOR ISS DESC. COND:

R$ 1.044.684,55 DEDUGÃO: R$0 CÁLCULO: 2% R$ 0,00 RETIDO: R$ 0,00
R$ 0,00 R$ 1.044.684,55 R$ 20.893,69
VALOR PIS: VALOR COFINS: VALOR IR: VALOR INSS: VALOR CSLL: OUTRAS RETENÇÕES: VALOR LÍQUIDO:

R$ 0,00 R$ 0,00 R$ 0,00 R$11491,53  R$0,00 R$ 0,00 R$ 1.012.299,33



In [ ]:
# Example of adding any additional options
custom_oem_psm_config = r'--oem 3 --psm 6'
pytesseract.image_to_string(image, config=custom_oem_psm_config)

# Example of using pre-defined tesseract config file with options
cfg_filename = 'words'
pytesseract.run_and_get_output(image, extension='txt', config=cfg_filename)

### 1. Parametros

In [6]:
dir_pattern = "novo_modelo/modelos/user-patterns2.txt"

In [7]:
# If you don't have tesseract executable in your PATH, include the following:
tessdata_dir_config_novo = '--tessdata-dir "/home/dani-boy/miniconda3/envs/tables-detr/share/tessdata/" --user-patterns "novo_modelo/modelos/user-patterns2.txt" --dpi 600 --oem 1 --psm 6'

In [ ]:
texto_celula = pytesseract.image_to_string(image, config=tessdata_dir_config_novo, lang='por')

In [ ]:
texto_celula

In [ ]:
print(texto_celula)

In [ ]:
image_path = "novo_modelo/images/processadas/sections/Captura de tela 2023-08-22 145853.png"

In [8]:
image_path = "novo_modelo/images/processadas/sections/6_frame_valores_impostos.jpg"

In [ ]:
image = Image.open(image_path).convert("RGB")

In [ ]:
width, height = image.size

In [ ]:
image.verify

In [ ]:
factor = 0.5
image = image.resize((int(width*factor), int(height*factor)))

In [ ]:
image

### Table Transformer

In [9]:
# Sera deprecated
from transformers import DetrFeatureExtractor
import torch




In [ ]:
encoding = feature_extractor(image, return_tensors="pt")
encoding.keys()

In [ ]:
with torch.no_grad():
  outputs = model(**encoding)

In [ ]:
# rescale bounding boxes
width, height = image.size
results = feature_extractor.post_process_object_detection(outputs, threshold=0.6, target_sizes=[(height, width)])[0]

In [ ]:
plot_results(image, results['scores'], results['labels'], results['boxes'])

In [ ]:
for roots, directories, images in os.walk(root_dir_section_images):
                for image in images:
                    #print(f'\n{roots}, {directories}, {images}')
                    #image_path = os.path.join(roots, image)
                    #print(image_path, image)
                    frame = image
                    
                    if frame == "6_frame_valores_impostos.jpg": 
                        image_path = os.path.join(roots, image)
                        image_to_ocr = Image.open(image_path).convert('RGB')
                        text = pytesseract.image_to_string(image_to_ocr, lang='por', config=tessdata_dir_config)
                           

tessdata_dir_config_novo = '--tessdata-dir "C:\\Program Files\\Tesseract-OCR\\tessdata" --dpi 300 --oem 1 --psm 4'

In [11]:
def processaOcrTable2(file_path, results):
    dados_da_tabela = {}
    # Carregue a imagem do PDF (ou outra fonte)
    img_pdf = cv2.imread(file_path, cv2.IMREAD_COLOR) 
    
    # Processo de OCR para cada célula e preenchimento do dicionário
    for i in range(results['boxes'].shape[0]):
        retangulo = results['boxes'][i].tolist()
        img_celula = recortar_imagem(img_pdf, retangulo)
        texto_celula = pytesseract.image_to_string(img_celula, config=tessdata_dir_config_novo, lang='por')
        
        # Mapear o ID numérico do rótulo para o rótulo descritivo usando model.config.id2label
        label_id = results['labels'][i].item() if isinstance(results['labels'][i], torch.Tensor) else results['labels'][i]
        label_descritivo = model.config.id2label[label_id]
        
        # Adicionar as informações ao dicionário
        chave_celula = f'celula_{i}'
        
        dados_da_tabela[chave_celula] = {
            'coordenadas': retangulo,
            'label': label_descritivo,
            'texto': texto_celula
        }
    return dados_da_tabela 

def processaTabela2(file_path, factor, t_hold):
  
  image = Image.open(file_path).convert("RGB")
  width, height = image.size
  image = image.resize((int(width*factor), int(height*factor)))

  feature_extractor = DetrImageProcessor()
  encoding = feature_extractor(image, return_tensors="pt")
  
  with torch.no_grad():
    outputs = model(**encoding)
      
  target_sizes = [image.size[::-1]]
  results = feature_extractor.post_process_object_detection(outputs, threshold=t_hold, target_sizes=target_sizes)[0]
  
  return results   



def plot_results(pil_img, scores, labels, boxes):
    plt.figure(figsize=(26,20))
    plt.imshow(pil_img)
    ax = plt.gca()
    colors = COLORS * 100
    for score, label, (xmin, ymin, xmax, ymax),c  in zip(scores.tolist(), labels.tolist(), boxes.tolist(), colors):
        ax.add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                   fill=False, color=c, linewidth=3))
        text = f'{model.config.id2label[label]}: {score:0.2f}'
        ax.text(xmin, ymin, text, fontsize=10,
                bbox=dict(facecolor='yellow', alpha=0.3))
    plt.axis('off')
    plt.show()

In [ ]:
image_path = "novo_modelo/images/processadas/sections/Captura de tela 2023-08-22 145853.png"

In [12]:
image_path

'novo_modelo/images/processadas/sections/6_frame_valores_impostos.jpg'

In [13]:
factor = 0.5
t_hold = 0.6

results = processaTabela2(image_path, factor, t_hold)

In [14]:
# 7. Processa OCR
dados_da_tabela = processaOcrTable2(image_path, results)

In [17]:
dados_da_tabela

{'celula_0': {'coordenadas': [482.7177734375,
   16.424301147460938,
   570.324462890625,
   130.5030059814453],
  'label': 'table column',
  'texto': 'RVIÇOS\n84,55\n'},
 'celula_1': {'coordenadas': [681.6405639648438,
   16.693172454833984,
   768.5257568359375,
   130.37234497070312],
  'label': 'table column',
  'texto': 'ÇÃO:\no\n'},
 'celula_2': {'coordenadas': [520.3229370117188,
   16.28337287902832,
   594.4053955078125,
   130.77102661132812],
  'label': 'table column',
  'texto': 'ços:\n5\n'},
 'celula_3': {'coordenadas': [591.4642333984375,
   16.514738082885742,
   672.9388427734375,
   130.43788146972656],
  'label': 'table column',
  'texto': 'VALO\nDEDL\nR$0,\n'},
 'celula_4': {'coordenadas': [54.0112190246582,
   17.287837982177734,
   160.11297607421875,
   130.41624450683594],
  'label': 'table column',
  'texto': 'g\n'},
 'celula_5': {'coordenadas': [407.39788818359375,
   16.63365364074707,
   483.12017822265625,
   130.44577026367188],
  'label': 'table column',
 

In [20]:
# 8. Captura dados (Regex)
textos = []
for i in dados_da_tabela:
    texto = dados_da_tabela[i]['texto']
    textos.append(texto)

In [18]:
# 8. Captura dados (Regex)
textos = []
for i in dados_da_tabela:
   if dados_da_tabela[i]['label'] == "table column":
      texto = dados_da_tabela[i]['texto']
      textos.append(texto)

In [21]:
textos

['RVIÇOS\n84,55\n',
 'ÇÃO:\no\n',
 'ços:\n5\n',
 'VALO\nDEDL\nR$0,\n',
 'g\n',
 'OR SE\n1.044.6;\n',
 'VAI\n| R$\n-i\n',
 'VALOR SERVIÇOS: VALOR | DESC. INCOND\num mm R$ 1.044.684,55 DEDUÇÃO: R$ 0,00\n',
 'DES:\nR$0\n',
 'Pe\n',
 '>. INCOND:\n00\n',
 'E tada R$ 0,00\na\n',
 'VALOR SERVIÇOS: VALOR | DESC. INCOND\num mm R$ 1.044.684,55 DEDUÇÃO: R$ 0,00\n',
 'Dies\n',
 'VALOR SERVIÇOS: VALOR | DESC. INCOND\nR$ 1.044.684,55 DEDUÇÃO: R$ 0,00\n-—\n']

In [22]:
print(textos)

['RVIÇOS\n84,55\n', 'ÇÃO:\no\n', 'ços:\n5\n', 'VALO\nDEDL\nR$0,\n', 'g\n', 'OR SE\n1.044.6;\n', 'VAI\n| R$\n-i\n', 'VALOR SERVIÇOS: VALOR | DESC. INCOND\num mm R$ 1.044.684,55 DEDUÇÃO: R$ 0,00\n', 'DES:\nR$0\n', 'Pe\n', '>. INCOND:\n00\n', 'E tada R$ 0,00\na\n', 'VALOR SERVIÇOS: VALOR | DESC. INCOND\num mm R$ 1.044.684,55 DEDUÇÃO: R$ 0,00\n', 'Dies\n', 'VALOR SERVIÇOS: VALOR | DESC. INCOND\nR$ 1.044.684,55 DEDUÇÃO: R$ 0,00\n-—\n']


In [ ]:
# 8. Captura dados (Regex)
textos = []
for i in dados_da_tabela:
   if dados_da_tabela[i]['label'] == "table column":
      texto = dados_da_tabela[i]['texto']
      textos.append(texto)
      #print(f"\n{i}: \n{dados_da_tabela[i]['texto']}")

In [ ]:
len(textos)

In [ ]:
textos

In [ ]:
import re

# Lista com os valores extraídos
valores_extraidos = [
    'ID: BASE DE\nCALCULO:\nR$ 1.044.684,5\nVALOR INSS:\nR$ 11.491,53\n',
    'VALOR ISS:\n\nR$ 0,00\nOUTRAS RETEN\nR$ 0,00\n',
    'DE Al\nJLO: 2\n44.684,55\n\n? INSS: V\n491,53 R\n',
    'ALÍQUOTA:\n2%\n\nVALOR CSLL:\nR$ 0,00\n',
    'a\nnjsiais\n',
    'C. INCOND:\n+00\n\nRIR:\n\n00\n',
    'DES:\nR$0\nFINS: VALC\nR$0.\n',
    'VALOR ISS\nRETIDO:\n\nR$ 20.893,69\nÇÕES:\n',
    '= VALOR\nDEDUÇÃO:\nR$ 0,00\nVALOR COF\nR$ 0.00\n',
    'DESC. COND:\nR$ 0,00\n\nVALOR LÍQUIDO:\nR$ 1.012.299,33\n',
    'VALOR SERVIÇOS:\n| R$ 1.044.684,55\n]\nJ\n«VALOR PIS:\n" R$000\n'
]

# Dicionário para associar chaves aos valores
valores_associados = {}

# Padrão de regex para valores monetários
padrao_monetario = r'R\$ ([\d,.]+)'

# Associar os valores extraídos às chaves
chave_atual = None
for texto in valores_extraidos:
    valores_encontrados = re.findall(padrao_monetario, texto)
    
    # Associar valores à chave atual
    for valor in valores_encontrados:
        if "VALOR SERVIÇOS":
            valor_formatado = valor.replace('.', '').replace(',', '.')
            valores_associados["VALOR SERVIÇOS"] = valor_formatado
            chave_atual = None
        else:
            chave_atual = valor

# Imprimir os valores associados
for chave, valor in valores_associados.items():
    print(f"{chave}: {valor}")



In [ ]:
import re

# Lista com os valores extraídos
valores_extraidos = [
    'ID: BASE DE\nCALCULO:\nR$ 1.044.684,5\nVALOR INSS:\nR$ 11.491,53\n',
    'VALOR ISS:\n\nR$ 0,00\nOUTRAS RETEN\nR$ 0,00\n',
    'DE Al\nJLO: 2\n44.684,55\n\n? INSS: V\n491,53 R\n',
    'ALÍQUOTA:\n2%\n\nVALOR CSLL:\nR$ 0,00\n',
    'a\nnjsiais\n',
    'C. INCOND:\n+00\n\nRIR:\n\n00\n',
    'DES:\nR$0\nFINS: VALC\nR$0.\n',
    'VALOR ISS\nRETIDO:\n\nR$ 20.893,69\nÇÕES:\n',
    '= VALOR\nDEDUÇÃO:\nR$ 0,00\nVALOR COF\nR$ 0.00\n',
    'DESC. COND:\nR$ 0,00\n\nVALOR LÍQUIDO:\nR$ 1.012.299,33\n',
    'VALOR SERVIÇOS:\n| R$ 1.044.684,55\n]\nJ\n«VALOR PIS:\n" R$000\n'
]

# Lista de chaves esperadas
chaves_esperadas = [
    'VALOR SERVIÇOS', 'VALOR DEDUÇÃO', 'DESC. INCOND', 'BASE DE CÁLCULO',
    'ALÍQUOTA', 'VALOR ISS', 'VALOR ISS RETIDO', 'VALOR CSLL', 'VALOR PIS',
    'VALOR COFINS', 'DESC. COND', 'VALOR LÍQUIDO'
]

# Dicionário para associar chaves aos valores
valores_associados = {}

# Padrão de regex para valores monetários
padrao_monetario = r'R\$ ([\d,.]+)'

# Associar os valores extraídos às chaves
chave_atual = None
for texto in valores_extraidos:
    valores_encontrados = re.findall(padrao_monetario, texto)
    
    # Associar valores à chave atual
    for valor in valores_encontrados:
        if chave_atual:
            valor_formatado = valor.replace('.', '').replace(',', '.')
            # Procurar por substrings nas chaves esperadas
            chave_encontrada = next((chave for chave in chaves_esperadas if chave in chave_atual), None)
            if chave_encontrada:
                valores_associados[chave_encontrada] = valor_formatado
            chave_atual = None
        else:
            chave_atual = valor

# Imprimir os valores associados
for chave, valor in valores_associados.items():
    print(f"{chave}: {valor}")


In [ ]:
import re

# Lista com os valores extraídos
valores_extraidos = [
    'ID: BASE DE\nCALCULO:\nR$ 1.044.684,5\nVALOR INSS:\nR$ 11.491,53\n',
    'VALOR ISS:\n\nR$ 0,00\nOUTRAS RETEN\nR$ 0,00\n',
    'DE Al\nJLO: 2\n44.684,55\n\n? INSS: V\n491,53 R\n',
    'ALÍQUOTA:\n2%\n\nVALOR CSLL:\nR$ 0,00\n',
    'a\nnjsiais\n',
    'C. INCOND:\n+00\n\nRIR:\n\n00\n',
    'DES:\nR$0\nFINS: VALC\nR$0.\n',
    'VALOR ISS\nRETIDO:\n\nR$ 20.893,69\nÇÕES:\n',
    '= VALOR\nDEDUÇÃO:\nR$ 0,00\nVALOR COF\nR$ 0.00\n',
    'DESC. COND:\nR$ 0,00\n\nVALOR LÍQUIDO:\nR$ 1.012.299,33\n',
    'VALOR SERVIÇOS:\n| R$ 1.044.684,55\n]\nJ\n«VALOR PIS:\n" R$000\n'
]

# Lista de chaves esperadas
chaves_esperadas = [
    'VALOR SERVIÇOS', 'VALOR DEDUÇÃO', 'DESC. INCOND', 'BASE DE CÁLCULO',
    'ALÍQUOTA', 'VALOR ISS', 'VALOR ISS RETIDO', 'VALOR CSLL', 'VALOR PIS',
    'VALOR COFINS', 'DESC. COND', 'VALOR LÍQUIDO'
]

# Dicionário para associar chaves aos valores
valores_associados = {}

# Padrão de regex para valores monetários
padrao_monetario = r'R\$ ([\d,.]+)'

# Associar os valores extraídos às chaves
chave_atual = None
for texto in valores_extraidos:
    valores_encontrados = re.findall(padrao_monetario, texto)
    
    # Associar valores à chave atual
    for valor in valores_encontrados:
        if chave_atual:
            valor_formatado = valor.replace('.', '').replace(',', '.')
            # Procurar por substrings nas chaves esperadas
            chave_encontrada = next((chave for chave in chaves_esperadas if chave in chave_atual), None)
            if chave_encontrada:
                valores_associados[chave_encontrada] = valor_formatado
            chave_atual = None
        else:
            chave_atual = valor

# Imprimir os valores associados
for chave, valor in valores_associados.items():
    print(f"{chave}: {valor}")


In [ ]:
VALOR SERVIÇOS: 


VALOR
DEDUÇÃO:


DESC. INCOND:

BASE DE
CÁLCULO:


ALÍQUOTA:


VALOR ISS:


VALOR ISS
RETIDO:


DESC. COND:


VALOR PIS:


VALOR COFINS:


VALOR INSS:


VALOR CSLL:


OUTRAS RETENÇÕES:


VALOR LÍQUIDO:


In [ ]:
import re


valores_extraidos = [
    'ID: BASE DE\nCALCULO:\nR$ 1.044.684,5\nVALOR INSS:\nR$ 11.491,53\n',
    'VALOR ISS:\n\nR$ 0,00\nOUTRAS RETEN\nR$ 0,00\n',
    'DE Al\nJLO: 2\n44.684,55\n\n? INSS: V\n491,53 R\n',
    'ALÍQUOTA:\n2%\n\nVALOR CSLL:\nR$ 0,00\n',
    'a\nnjsiais\n',
    'C. INCOND:\n+00\n\nRIR:\n\n00\n',
    'DES:\nR$0\nFINS: VALC\nR$0.\n',
    'VALOR ISS\nRETIDO:\n\nR$ 20.893,69\nÇÕES:\n',
    '= VALOR\nDEDUÇÃO:\nR$ 0,00\nVALOR COF\nR$ 0.00\n',
    'DESC. COND:\nR$ 0,00\n\nVALOR LÍQUIDO:\nR$ 1.012.299,33\n',
    'VALOR SERVIÇOS:\n| R$ 1.044.684,55\n]\nJ\n«VALOR PIS:\n" R$000\n'
]



# Texto completo com os valores extraídos
texto_completo = '\n'.join(valores_extraidos)

# Lista de chaves esperadas
chaves_esperadas = [
    'BASE DE\nCALCULO:',  #BASE DE CÁLCULO:R$ 1.044.684,55
    'VALOR INSS:', 
    'VALOR ISS:',
    'OUTRAS RETEN',
    'ALÍQUOTA:',
    'VALOR CSLL:'
]

# Dicionário para associar chaves aos valores
valores_associados = {}

# Padrão de regex para valores monetários e percentuais
padrao_monetario_percentual = r'([\d.,]+)(%|\s?R\$)'

# Encontrar e associar os valores
chave_atual = None
for linha in valores_extraidos:
    linha = linha.replace('\n', ' ')  # Remover quebras de linha
    valores_encontrados = re.findall(padrao_monetario_percentual, linha)
    
    for valor, unidade in valores_encontrados:
        if chave_atual:
            chave_encontrada = next((chave for chave in chaves_esperadas if chave.lower() in chave_atual.lower()), None)
            if chave_encontrada:
                if unidade == '%':
                    valor_formatado = f"{valor}%"
                else:
                    valor_formatado = valor.replace('.', '').replace(',', '.')
                    valor_formatado = f"R$ {valor_formatado}"
                valores_associados[chave_encontrada] = valor_formatado
            chave_atual = None
        else:
            chave_atual = valor

# Imprimir os valores associados
for chave, valor in valores_associados.items():
    print(f"{chave}: {valor}")

In [ ]:
import re


valores_extraidos = [
    'ID: BASE DE\nCALCULO:\nR$ 1.044.684,5\nVALOR INSS:\nR$ 11.491,53\n',
    'VALOR ISS:\n\nR$ 0,00\nOUTRAS RETEN\nR$ 0,00\n',
    'DE Al\nJLO: 2\n44.684,55\n\n? INSS: V\n491,53 R\n',
    'ALÍQUOTA:\nR$ 2,00\n\nVALOR CSLL:\nR$ 0,00\n',
    'a\nnjsiais\n',
    'C. INCOND:\n+00\n\nRIR:\n\n00\n',
    'DES:\nR$0\nFINS: VALC\nR$0.\n',
    'VALOR ISS\nRETIDO:\n\nR$ 20.893,69\nÇÕES:\n',
    '= VALOR\nDEDUÇÃO:\nR$ 0,00\nVALOR COF\nR$ 0.00\n',
    'DESC. COND:\nR$ 0,00\n\nVALOR LÍQUIDO:\nR$ 1.012.299,33\n',
    'VALOR SERVIÇOS:\n| R$ 1.044.684,55\n]\nJ\n«VALOR PIS:\n" R$000\n'
]



# Texto completo com os valores extraídos
texto_completo = '\n'.join(valores_extraidos)

# Lista de chaves esperadas
chaves_esperadas = [
    'BASE DE\nCALCULO',  #BASE DE CÁLCULO:R$ 1.044.684,55
    'VALOR INSS', 
    'VALOR ISS',
    'OUTRAS RETEN',
    'ALÍQUOTA',
    'VALOR CSLL',
    'VALOR\nDEDUÇÃO',
    'VALOR ISS\nRETIDO',
    'VALOR SERVIÇOS',
    'VALOR PIS'
]

# Dicionário para associar chaves aos valores
valores_associados = {}

# Padrão de regex para valores monetários
padrao_monetario = r'R\$ ([\d,.]+)'

padrao = r'([\d.,]+)(%|\s?R\$)'

# Encontrar e associar os valores
for chave in chaves_esperadas:
    valor_encontrado = re.search(f'{chave}.*?{padrao_monetario}', texto_completo, re.DOTALL)
    if valor_encontrado:
        valor_formatado = valor_encontrado.group(1).replace('.', '').replace(',', '.')
        valores_associados[chave] = valor_formatado

# Imprimir os valores associados
for chave, valor in valores_associados.items():
    print(f"{chave}: {valor}")


In [ ]:
BASE DE
CÁLCULO:
R$ 1.044.684,55

VALOR SERVIÇOS: 
R$ 1.044.684,55

VALOR INSS:
R$ 11.491,53

VALOR ISS:
R$ 0,00

OUTRAS RETENÇÕES:
R$ 0,00

ALÍQUOTA:
2%

VALOR CSLL:
R$ 0,00

VALOR
DEDUÇÃO:
R$ 0,00


VALOR ISS
RETIDO:
R$ 20.893,69

In [ ]:
    'VALOR DEDUÇÃO', 'DESC. INCOND', 'BASE DE CÁLCULO',
    'ALÍQUOTA', 'VALOR ISS', 'VALOR ISS RETIDO', 'VALOR CSLL', 'VALOR PIS',
    'VALOR COFINS', 'DESC. COND', 'VALOR LÍQUIDO', 'DEDUÇÃO:

In [ ]:




DESC. INCOND:
R$ 0,00








DESC. COND:
R$ 0,00

VALOR PIS:
R$ 0,00

VALOR COFINS:
R$ 0,00







VALOR LÍQUIDO:
R$ 1.012.299,33

In [ ]:
import re

# Lista com os valores extraídos
valores_extraidos = [
    'ID: BASE DE\nCALCULO:\nR$ 1.044.684,5\nVALOR INSS:\nR$ 11.491,53\n',
    'VALOR ISS:\n\nR$ 0,00\nOUTRAS RETEN\nR$ 0,00\n',
    'DE Al\nJLO: 2\n44.684,55\n\n? INSS: V\n491,53 R\n',
    'ALÍQUOTA:\n2%\n\nVALOR CSLL:\nR$ 0,00\n',
    'a\nnjsiais\n',
    'C. INCOND:\n+00\n\nRIR:\n\n00\n',
    'DES:\nR$0\nFINS: VALC\nR$0.\n',
    'VALOR ISS\nRETIDO:\n\nR$ 20.893,69\nÇÕES:\n',
    '= VALOR\nDEDUÇÃO:\nR$ 0,00\nVALOR COF\nR$ 0.00\n',
    'DESC. COND:\nR$ 0,00\n\nVALOR LÍQUIDO:\nR$ 1.012.299,33\n',
    'VALOR SERVIÇOS:\n| R$ 1.044.684,55\n]\nJ\n«VALOR PIS:\n" R$000\n'
]

# Lista de chaves esperadas
chaves_esperadas = [
    'VALOR SERVIÇOS:', 'VALOR DEDUÇÃO', 'DESC. INCOND', 'ID: BASE DE\nCALCULO:',
    'ALÍQUOTA', 'VALOR ISS', 'VALOR ISS RETIDO', 'VALOR CSLL', 'VALOR PIS',
    'VALOR COFINS', 'DESC. COND', 'VALOR LÍQUIDO'
]

# Dicionário para associar chaves aos valores
valores_associados = {}

# Padrão de regex para valores monetários
padrao_monetario = r'R\$ ([\d.,]+)'

# Encontrar e associar os valores
chave_atual = None
for linha in valores_extraidos:
    linha = linha.replace('\n', ' ')  # Remover quebras de linha
    valores_encontrados = re.findall(padrao_monetario, linha)
    
    for valor in valores_encontrados:
        if chave_atual:
            chave_encontrada = next((chave for chave in chaves_esperadas if chave in chave_atual), None)
            if chave_encontrada:
                valor_formatado = valor.replace('.', '').replace(',', '.')
                valores_associados[chave_encontrada] = valor_formatado
            chave_atual = None
        else:
            chave_atual = valor

# Imprimir os valores associados
for chave, valor in valores_associados.items():
    print(f"{chave}: {valor}")


In [ ]:
import re

# Lista com os valores extraídos
valores_extraidos = [
    'ID: BASE DE\nCALCULO:\nR$ 1.044.684,5\nVALOR INSS:\nR$ 11.491,53\n',
    'VALOR ISS:\n\nR$ 0,00\nOUTRAS RETEN\nR$ 0,00\n',
    'DE Al\nJLO: 2\n44.684,55\n\n? INSS: V\n491,53 R\n',
    'ALÍQUOTA:\n2%\n\nVALOR CSLL:\nR$ 0,00\n',
    'a\nnjsiais\n',
    'C. INCOND:\n+00\n\nRIR:\n\n00\n',
    'DES:\nR$0\nFINS: VALC\nR$0.\n',
    'VALOR ISS\nRETIDO:\n\nR$ 20.893,69\nÇÕES:\n',
    '= VALOR\nDEDUÇÃO:\nR$ 0,00\nVALOR COF\nR$ 0.00\n',
    'DESC. COND:\nR$ 0,00\n\nVALOR LÍQUIDO:\nR$ 1.012.299,33\n',
    'VALOR SERVIÇOS:\n| R$ 1.044.684,55\n]\nJ\n«VALOR PIS:\n" R$000\n'
]

# Lista de chaves esperadas
chaves_esperadas = [
    'VALOR SERVIÇOS', 'VALOR DEDUÇÃO', 'DESC. INCOND', 'BASE DE CÁLCULO',
    'ALÍQUOTA', 'VALOR ISS', 'VALOR ISS RETIDO', 'VALOR CSLL', 'VALOR PIS',
    'VALOR COFINS', 'DESC. COND', 'VALOR LÍQUIDO'
]

# Dicionário para associar chaves aos valores
valores_associados = {}

# Padrão de regex para valores monetários
padrao_monetario = r'R\$ ([\d.,]+)'

# Encontrar e associar os valores
chave_atual = None
for linha in valores_extraidos:
    linha = linha.replace('\n', ' ')  # Remover quebras de linha
    valores_encontrados = re.findall(padrao_monetario, linha)
    
    for valor in valores_encontrados:
        valor_formatado = valor.replace('.', '').replace(',', '.')
        if chave_atual:
            # Tentar associar a chave atual às chaves esperadas
            for chave_esperada in chaves_esperadas:
                if chave_esperada.lower() in chave_atual.lower():
                    valores_associados[chave_esperada] = valor_formatado
                    break
            chave_atual = None
        else:
            chave_atual = valor

# Imprimir os valores associados
for chave, valor in valores_associados.items():
    print(f"{chave}: {valor}")



In [ ]:
linhas = []
i = 0
nro_lin = len(linhas)
for i in range(nro_lin):
    print(i)
    #novo_texto = textos[i]
    #print(i, novo_texto)
    #novo_texto = texto.split('\n')
    #linhas.append(novo_texto)

In [ ]:
len(linhas)

In [ ]:
linhas

In [ ]:
# Dividir o texto em linhas
linhas = textos.split('\n')

In [ ]:


# Inicializar variáveis para armazenar os valores
nome_razao_social = None
endereco = None
email = None

# Iterar pelas linhas para identificar os campos e valores
i = 0
while i < len(linhas):
    linha = linhas[i]
    if "Nome/Razão Social:" in linha:
        nome_razao_social = linhas[i + 1].strip()
        i += 2
    elif "Endereço:" in linha:
        endereco = linhas[i + 1].strip()
        i += 2
    elif "E-mail:" in linha:
        email = linhas[i + 1].strip()
        i += 2
    else:
        i += 1

In [ ]:
for texto in textos:
    print(texto)

In [ ]:
results

### Outros pedacos

In [ ]:
nf_data_CNAE = {}
nf_data_CNAE['Secao'] = "6. CNAE e Item da Lista de Serviços"

linhas = text.split('º\n')


# Extrair CNAE
nf_data_CNAE_match = re.search(r'CNAE\s+(.+)', text)
if nf_data_CNAE_match:
    # Remove a primeira ocorrência de "CNAE:"
    nf_data_CNAE_str = re.sub(r'^CNAE - ', '', text, count=1)
    # Remover quebras de linha
    nf_data_CNAE_str = nf_data_CNAE_str.replace('\n', ' ')
    nf_data_CNAE['cnae'] = nf_data_CNAE_str
    
    
# Extrair Item da Lista de Serviços
nf_item_lista_servicos_match = re.search(r'Item da Lista de Serviços\s+(.+)', text)
if nf_item_lista_servicos_match:
    nf_item_lista_servicos_str = re.sub(r'^Item da Lista de Serviços - ', '', text, count=1) 
    # Remover quebras de linha
    #nf_item_lista_servicos_str = nf_item_lista_servicos_str.replace('\n \n', '')
    nf_item_lista_servicos_str = nf_item_lista_servicos_str.replace('\n', ' ')
    nf_data_CNAE['item_lista_servicos'] = nf_item_lista_servicos_str   
    
    
    

In [ ]:
linhas[0].replace(' . .', '')

In [ ]:
texto_1 = linhas[1]

In [ ]:
texto_1

In [ ]:
texto_novo = texto_novo.replace('\n', '')

In [ ]:
item_lista_servico = re.sub(r'^Item da Lista de Serviços - ', '', texto_novo, count=1)

In [ ]:
item_lista_servico

In [ ]:
nf_data_CNAE

In [ ]:
nf_data_CNAE = {}
nf_data_CNAE['Secao'] = "6. CNAE e Item da Lista de Serviços"


linhas = text.split('º\n')

"""if "\n\n" in text:
    linhas = text.split('\n\n')
    tipo = 2
    
else:
    linhas = text.split('\n')    
    tipo = 1"""



# Inicializar variáveis para armazenar os valores
cnae = None
item_lista_servico = None

texto1 = None
texto2 = None

# Iterar pelas linhas para identificar os campos e valores
i = 0
while i < len(linhas):
    linha = linhas[i]
    #print("linha:", i, linha)
    if "CNAE" in linha:
        texto1 = linhas[i].strip().replace('º', '.').replace(' . .', '')
        cnae = re.sub(r'^CNAE - ', '', texto1, count=1)
        i += 1
               
        print("CNAE:",cnae)
        
        #print()
        #print("texto_razao2: ", texto_razao2)
        #print()
        #print("texto_razao3: ", texto_razao3)
        #print(f'\nRazao Social: {nome_razao_social}')
        
    elif "Item da Lista de Serviços" in linha:
        
        #print("Aqui")
        texto1 = linhas[i].strip()
        #texto2 = linhas[i + 1].strip()
        
        print("texto1:",texto1)
        
        print()
        
        #print("texto2:",texto2)
        
        #)
        
        
        
        #texto2 = re.sub(r'^Item da Lista de Serviços - ', '', texto1, count=1)
        
        #item_lista_servico = texto2
        
        
        i += 2  
        #print("Aqui - item_lista_servico: ",item_lista_servico)
        
    else:
        i += 1


# Imprimir os valores dos campos
#print("Nome/Razão Social:", nome_razao_social)

#print("item_lista_servico:",item_lista_servico)

#nf_data_prestador_servico['razao_social'] = nome_razao_social

In [ ]:
linhas

In [ ]:
nf_data_CNAE = {}
nf_data_CNAE['Secao'] = "6. CNAE e Item da Lista de Serviços"


nf_data_CNAE_match = re.search(r'CNAE\s+(.+)', text)
if nf_data_CNAE_match:
    # Remove a primeira ocorrência de "CNAE:"
    nf_data_CNAE_str = re.sub(r'^CNAE - ', '', text, count=1)
    # Remover quebras de linha
    nf_data_CNAE_str = nf_data_CNAE_str.replace('\n', ' ')
    nf_data_CNAE['cnae'] = nf_data_CNAE_str


nf_item_lista_servicos_match = re.search(r'Item da Lista de Serviços\s+(.+)', text)
if nf_item_lista_servicos_match:
    nf_item_lista_servicos_str = re.sub(r'^Item da Lista de Serviços - ', '', text, count=1) 
    # Remover quebras de linha
    #nf_item_lista_servicos_str = nf_item_lista_servicos_str.replace('\n \n', '')
    nf_item_lista_servicos_str = nf_item_lista_servicos_str.replace('\n', ' ')
    nf_data_CNAE['item_lista_servicos'] = nf_item_lista_servicos_str




In [ ]:
nf_data_CNAE

In [ ]:
# Prestador de servico - Razao social / Endereco / Nome Fantasia / email
text = 'Nome/Razão Social:\n\nSÃO MARCOS TERRAPLENAGEM E CONSTRUÇÃO LTDA.\nNome de Fantasia:\n\nEndereço:\n\nRUA DAS MARGARIDAS ,578 ,SANTA DALILA - Magé-RJ\nE-mail:\n'

nf_data_prestador_servico = {}
nf_data_prestador_servico['secao'] = "2.PRESTADOR DE SERVIÇO - RAZAO"
tipo = None

if "\n\n" in text:
    linhas = text.split('\n\n')
    tipo = 2
    
else:
    linhas = text.split('\n')    
    tipo = 1
   
   
#linhas = text.split('\n')

#linhas = text.split('\n\n')
#print(linhas)


# Inicializar variáveis para armazenar os valores
nome_razao_social = None
nome_fantasia = None
endereco = None
email = None

# Iterar pelas linhas para identificar os campos e valores
i = 0
while i < len(linhas):
    linha = linhas[i]
    #print("linha:", i, linha)
    if "Nome/Razão Social:" in linha:
        if tipo == 2:
            texto1 = linhas[i + 1].strip()
            texto2 = texto1.split('\n')
            nome_razao_social = texto2[i].strip()
        else:
            nome_razao_social = linhas[i + 1].strip()   
        #print("texto_razao: ", texto_razao)
        
        #print()
        #print("texto_razao2: ", texto_razao2)
        #print()
        #print("texto_razao3: ", texto_razao3)
        #print(f'\nRazao Social: {nome_razao_social}')
        i += 2
    elif "Nome de Fantasia:" in linha:
        nome_fantasia = linhas[i + 1].strip()
        i += 2
    elif "Endereço:" in linha:
        
        if tipo == 2:
            texto3 = linhas[i + 1].strip()
            texto4 = texto3.split('\n')
            
            endereco = texto4[0].strip()
            print("endereco:", i, endereco)
            
            
        else:
            endereco = linhas[i + 1].strip()      
        i += 2
    elif "E-mail:" in linha:
        email = linhas[i + 1].strip()
        i += 2
    else:
        i += 1


# Imprimir os valores dos campos
print("Nome/Razão Social:", nome_razao_social)
print("Nome de Fantasia:", nome_fantasia)
print("Endereço:", endereco)
print("E-mail:", email)


nf_data_prestador_servico['razao_social'] = nome_razao_social
nf_data_prestador_servico['nome_fantasia'] = nome_fantasia
nf_data_prestador_servico['endereco'] = endereco
nf_data_prestador_servico['email'] = email

## 3. Processo

In [ ]:
#nf_model = "MI_1"

nf_model = "mage_1"

nf_data_servico = {}

pdf_info = {}  # Dicionário para armazenar informações sobre PDFs

for root, dirs, files in os.walk(root_directory):
    for file in files:
        file_path = os.path.join(root, file)

        if file.lower().endswith('.pdf'):
            #print(file_path)
                
            # 1. Conversao para imagem e resize
            converte2image(file_path)
                
            # 2. Resize
            resizeImage(image_src)
                
            # 3. Executar cortes
            #file_path = img_resi_path
            #image_to_crop = Image.open(img_resi_path).convert("RGB")
            #width, height = image_to_crop.size

            cropSections(nf_model, img_resi_path)
                
            for roots, directories, images in os.walk(root_dir_section_images):
                for image in images:
                    #print(f'\n{roots}, {directories}, {images}')
                    print(image)
                    image_path = os.path.join(roots, image)
                    #print(image_path, image)
                    frame = image
                    if frame == "0_frame_dados_nf.jpg":
                        nro_nota = 0
                        image_to_ocr = Image.open(image_path).convert('RGB')
                        text = pytesseract.image_to_string(image_to_ocr, lang='por', config=tessdata_dir_config)
                        nf_data_dados, nro_nota = extract_fields_dados(text)

                    if frame == "0_frame_prefeitura_nf.jpg":
                        image_to_ocr = Image.open(image_path).convert('RGB')
                        text = pytesseract.image_to_string(image_to_ocr, lang='por', config=tessdata_dir_config)
                        nf_data_prefeitura, nome_prefeitura  = extract_fields_prefeitura(text)    
                        
                    if frame == "1_frame_prestador_cnpj.jpg": 
                        image_to_ocr = Image.open(image_path).convert('RGB')
                        text = pytesseract.image_to_string(image_to_ocr, lang='por', config=tessdata_dir_config)
                        nf_data_prestador = extract_fields_prestador(text)
                        
                               
                    if frame == "1_frame_prestador_inscricao.jpgg": 
                        image_to_ocr = Image.open(image_path).convert('RGB')
                        text = pytesseract.image_to_string(image_to_ocr, lang='por', config=tessdata_dir_config)
                        nf_data_prestador = extract_fields_prestador(text)    
                            
                    if frame == "1_frame_prestador_servico.jpg": 
                        image_to_ocr = Image.open(image_path).convert('RGB')
                        text = pytesseract.image_to_string(image_to_ocr, lang='por', config=tessdata_dir_config)
                        nf_data_prestador = extract_fields_prestador(text)      
                        
                        
                        
                        
                        
                        
                        
                    if frame == "2_frame_tomador_cnpj.jpg": 
                        image_to_ocr = Image.open(image_path).convert('RGB')
                        text = pytesseract.image_to_string(image_to_ocr, lang='por', config=tessdata_dir_config)
                        nf_data_tomador_cnpj = extract_fields_tomador_cnpj(text)                                                       
                    
                    
                    if frame == "2_frame_tomador_inscricao.jpg": 
                        image_to_ocr = Image.open(image_path).convert('RGB')
                        text = pytesseract.image_to_string(image_to_ocr, lang='por', config=tessdata_dir_config)
                        nf_data_tomador_inscricao = extract_fields_tomador_inscricao(text)                            
                        
                        
                    if frame == "2_frame_tomador_servico.jpg": 
                        image_to_ocr = Image.open(image_path).convert('RGB')
                        text = pytesseract.image_to_string(image_to_ocr, lang='por', config=tessdata_dir_config)
                        nf_data_tomador = extract_fields_tomador(text)  
                            
                            
                            
                            # 6. Processa analise de tabela
                            #results = processaTabela(image_path)
                            
                            # 7. Processa OCR
                            #dados_da_tabela = processaOcrTable(image_path, results)
                            
                            #for i in dados_da_tabela:
                                #if dados_da_tabela[i]['label'] == "table column":
                                    #text = dados_da_tabela[i]['texto']
                                    #print(f"\n{i}: \n{dados_da_tabela[i]['texto']}")
                                    
                    nr_nro_nf = nro_nota
                        
                    nome_arquivo_json = os.path.basename(root) + ".json"
                    nome_arquivo = file
                                                
                    pdf_info[nr_nro_nf] = {
                        "dados_NF_PDF": {
                            "data_nf": nf_data_dados,
                            "data_prefeitura": nf_data_prefeitura,
                            "data_prestador": nf_data_prestador,
                            "data_tomador_cnpj": nf_data_tomador_cnpj,
                            "data_tomador_inscricao": nf_data_tomador_inscricao,
                            "data_tomador": nf_data_tomador,
                            },
                        "diretorio": os.path.basename(root),
                        "nome_arquivo": nome_arquivo,    
                    }            
                        
                        
# Salvando as informações em um arquivo JSON
json_file_path = os.path.join(target_directory, nome_arquivo_json)
with open(json_file_path, "w", encoding='utf-8') as json_file:
    json.dump(pdf_info, json_file, ensure_ascii=False, indent=4)                    
        


In [ ]:
print(text)

In [ ]:
nome_prefeitura

In [ ]:
nf_data_dados

In [ ]:
nf_data_prefeitura

In [ ]:
nf_data_prestador

In [ ]:
nf_data_tomador

In [ ]:
nf_data_tomador_cnpj

In [ ]:
nf_data_tomador_inscricao

In [ ]:
                nome_arquivo_json = os.path.basename(root) + ".json"
                nome_arquivo = file
                        
                pdf_info[nr_nro_nf] = {
                    "dados_NF_PDF": {
                        "data_cabecalho": nf_data_cabecalho,
                            
                    },
                    "diretorio": os.path.basename(root),
                    "nome_arquivo": nome_arquivo,    
                }            
                        
                        
# Salvando as informações em um arquivo JSON
json_file_path = os.path.join(target_directory, nome_arquivo_json)
with open(json_file_path, "w", encoding='utf-8') as json_file:
    json.dump(pdf_info, json_file, ensure_ascii=False, indent=4)          

In [ ]:
# 1. Conversao para imagem e resize
converte2image(file)

In [ ]:
# 2. Resize
resizeImage(image_src)

In [ ]:
# 4. Executar cortes
file_path = img_resi_path
image = Image.open(file_path).convert("RGB")
width, height = image.size

cropSections(nf_model, img_resi_path)

In [ ]:
# 5. Definir corte para mapeamento

#file_path = r".\novo_modelo\images\processadas\sections\0_frame_cabecalho_nf.jpg"

file_path = r".\novo_modelo\images\processadas\sections\1_frame_prestador_servico.jpg"

#file_path = r".\novo_modelo\images\processadas\sections\2_frame_tomador_servico.jpg"

#file_path = r".\novo_modelo\images\processadas\sections\3_frame_descricao_servico.jpg"

#file_path = r".\novo_modelo\images\processadas\sections\4_frame_valor_total.jpg"

#file_path = r".\novo_modelo\images\processadas\sections\5_frame_cnae_itens.jpg"

#file_path = r".\novo_modelo\images\processadas\sections\6_frame_valores_impostos.jpg"

#file_path = r".\novo_modelo\images\processadas\sections\7_frame_dados_complementares.jpg"

#file_path = r".\novo_modelo\images\processadas\sections\9_frame_observacao.jpg"

In [ ]:
# 6. Processa analise de tabela
results = processaTabela(file_path)

In [ ]:
# 7. Processa OCR
dados_da_tabela = processaOcrTable(file_path, results)

In [ ]:
# 8. Captura dados (Regex)

for i in dados_da_tabela:
   if dados_da_tabela[i]['label'] == "table column":
      print(f"\n{i}: \n{dados_da_tabela[i]['texto']}")

In [ ]:
# Salvando as informações em um arquivo JSON
#json_file_path = os.path.join(target_directory, "pdf_image_info.json")
with open("dados_da_tabela.json", "w") as json_file:
    json.dump(dados_da_tabela, json_file, indent=4)

#print(f"As informações foram salvas em {json_file_path}")

# Outros

In [ ]:

x0 = 510
y0 = 0
x1 = 1540
y1 = 380

section = "0_frame_prefeitura_nf"
defineDicionario(section, x0, y0, x1, y1)

# Efetuar o crop da imagem da NF segmentando por secoes
def defineDicionario(img_resized, section, x0, y0, x1, y1):
    im = Image.open(img_resized)
    left = x0
    top = y0
    right = x1
    bottom = y1
    im_croped_img = im.crop((left, top, right, bottom))
    img_file_path = os.path.join(f"{tgt_imagens}\processadas\sections\{section}.jpg")
    im_croped_img.save(img_file_path, 'JPEG') 
     






    for model_name, cuts in cut_dict.items():
        if model_name == nf_model:
            print(f"Modelo: {model_name}")
            for cut in cuts:
                model_section = cut['Item']
                x0 = cut['x0']
                y0 = cut['y0']
                x1 = cut['x1']
                y1 = cut['y1']
                cropDocumento(img_resi_path, model_section, x0, y0, x1, y1)
                
def cropDocumento(img_resized, section, x0, y0, x1, y1):
    im = Image.open(img_resized)
    left = x0
    top = y0
    right = x1
    bottom = y1
    im_croped_img = im.crop((left, top, right, bottom))
    #img_file_path = os.path.join(f"{tgt_imagens}\processadas\sections\{section}.jpg")
    #im_croped_img.save(img_file_path, 'JPEG')   
    text = pytesseract.image_to_string(im_croped_img, lang='por', config=tessdata_dir_config)
    nf_data_prefeitura, nome_prefeitura  = extract_fields_prefeitura(text)
    if nome_prefeitura ==  PREFEITURA MUNICIPAL DE MAGE:
         nf_model = "mage_1"
    elif
       nome_prefeitura ==  PREFEITURA MUNICIPAL DE MAGE:   

In [ ]:
print(f"\n| scores: {results['scores']} | labels: {results['labels']} | boxes: {results['boxes']}")

In [ ]:
for model_name, cuts in cut_dict.items():
    print(f"Modelo: {model_name}")
    for cut in cuts:
        print(f"  Item: {cut['Item']}, x0: {cut['x0']}, y0: {cut['y0']}, x1: {cut['x1']}, y1: {cut['y1']}")

In [ ]:
plot_results(image, results['scores'], results['labels'], results['boxes'])

model.config.id2label

In [ ]:
for roots, directories, images in os.walk(root_dir_section_images):
    for image in images:
        image_path = os.path.join(roots, image)
        print(image_path)

In [ ]:
print(dados_da_tabela['boxes'].shape)

In [ ]:
print(dados_da_tabela['labels'])

In [ ]:
print(dados_da_tabela['labels'])


In [ ]:
print(dados_da_tabela['labels'][0])


In [ ]:
dados_da_tabela.shape

In [ ]:
print(dados_da_tabela['labels'].shape)

In [ ]:
print(dados_da_tabela['pixel_values'][:, :])


In [ ]:
# Função para recortar a imagem
def recortar_imagem(img, coordenadas):
    x_min, y_min, x_max, y_max = coordenadas
    return img[y_min:y_max, x_min:x_max]

# Exemplo de uso da função
import numpy as np

# Criar uma imagem de exemplo
img_exemplo = np.zeros((800, 1000), dtype=np.uint8)  # Imagem preta de 1000x800 pixels

# Definir coordenadas para recortar uma região retangular na imagem
coordenadas_exemplo = [100, 200, 400, 500]

# Recortar a região da imagem
regiao_recortada = recortar_imagem(img_exemplo, coordenadas_exemplo)

# Imprimir a forma da região recortada
print(regiao_recortada.shape)


In [ ]:
# Processo de OCR para cada célula e preenchimento do dicionário
for i in range(dados_da_tabela['labels'].shape[0]):
    retangulo = dados_da_tabela['pixel_values'][i].tolist()
    img_celula = recortar_imagem(img_pdf, retangulo)
    texto_celula = pytesseract.image_to_string(img_celula, config=tesseract_config, lang='por')
    
    # Mapear o ID numérico do rótulo para o rótulo descritivo usando model.config.id2label
    label_id = dados_da_tabela['labels'][i].item() if isinstance(dados_da_tabela['labels'][i], torch.Tensor) else dados_da_tabela['labels'][i]
    label_descritivo = model.config.id2label[label_id]
    
    # Adicionar as informações ao dicionário
    chave_celula = f'celula_{i}'
    dados_da_tabela[chave_celula] = {
        'coordenadas': retangulo,
        'label': label_descritivo,
        'texto': texto_celula
    }

# Remover as chaves originais, pois foram substituídas pelas chaves com informações completas
chaves_originais = ['pixel_values', 'labels']
for chave_original in chaves_originais:
    dados_da_tabela.pop(chave_original)

# Agora o dicionário 'dados_da_tabela' contém informações completas de cada célula detectada
print(dados_da_tabela)


In [ ]:
print(dados_da_tabela)



In [ ]:
dados_da_tabela.shape

In [ ]:
labels_ids.shape

In [ ]:
# Processo de OCR para cada célula e preenchimento do dicionário
for i in range(dados_da_tabela.shape[0]):
    retangulo = dados_da_tabela[i, :].tolist()  # Converter o tensor para uma lista de coordenadas
    img_celula = recortar_imagem(file_path, retangulo)
    texto_celula = pytesseract.image_to_string(img_celula, config=tesseract_config, lang='por')
    
    # Mapear o ID numérico do rótulo para o rótulo descritivo usando model.config.id2label
    label_id = dados_da_tabela['labels'][i].item() if isinstance(dados_da_tabela['labels'][i], torch.Tensor) else dados_da_tabela['labels'][i]
    label_descritivo = model.config.id2label[label_id]
    
    # Adicionar as informações ao dicionário
    chave_celula = f'celula_{i}'
    dados_da_tabela[chave_celula] = {
        'coordenadas': retangulo,
        'label': label_descritivo,
        'texto': texto_celula
    }

# Remover as chaves originais, pois foram substituídas pelas chaves com informações completas
chaves_originais = ['pixel_values', 'labels']
for chave_original in chaves_originais:
    dados_da_tabela.pop(chave_original)

# Agora o dicionário 'dados_da_tabela' contém informações completas de cada célula detectada
print(dados_da_tabela)


In [ ]:
dados_da_tabela['labels']

In [ ]:
# Processo de OCR para cada célula e preenchimento do dicionário
for i in range(dados_da_tabela.shape[0]):
    retangulo = dados_da_tabela[i, :]
    img_celula = recortar_imagem(file_path, retangulo)
    texto_celula = pytesseract.image_to_string(img_celula, config=tesseract_config, lang='por')
    
    # Mapear o ID numérico do rótulo para o rótulo descritivo usando model.config.id2label
    label_id = dados_da_tabela['labels'][i].item() if isinstance(dados_da_tabela['labels'][i], torch.Tensor) else dados_da_tabela['labels'][i]
    label_descritivo = model.config.id2label[label_id]
    
    # Adicionar as informações ao dicionário
    chave_celula = f'celula_{i}'
    dados_da_tabela[chave_celula] = {
        'coordenadas': retangulo.tolist(),
        'label': label_descritivo,
        'texto': texto_celula
    }

# Remover as chaves originais, pois foram substituídas pelas chaves com informações completas
chaves_originais = ['pixel_values', 'labels']
for chave_original in chaves_originais:
    dados_da_tabela.pop(chave_original)

# Agora o dicionário 'dados_da_tabela' contém informações completas de cada célula detectada
print(dados_da_tabela)


In [ ]:
# Processo de OCR para cada célula e preenchimento do dicionário
#tesseract_config = '--oem 1'  # Defina as configurações específicas do Tesseract aqui, se necessário
for i in range(len(dados_da_tabela['pixel_values'])):
    retangulo = dados_da_tabela['pixel_values'][i]
    img_celula = recortar_imagem(file_path, retangulo)
    texto_celula = pytesseract.image_to_string(img_celula, config=tesseract_config, lang='por')
    
    # Mapear o ID numérico do rótulo para o rótulo descritivo usando model.config.id2label
    label_id = dados_da_tabela['labels'][i].item() if isinstance(dados_da_tabela['labels'][i], torch.Tensor) else dados_da_tabela['labels'][i]
    label_descritivo = model.config.id2label[label_id]
    
    # Adicionar as informações ao dicionário
    chave_celula = f'celula_{i}'
    dados_da_tabela[chave_celula] = {
        'coordenadas': retangulo.tolist(),
        'label': label_descritivo,
        'texto': texto_celula
    }

# Remover as chaves originais, pois foram substituídas pelas chaves com informações completas
chaves_originais = ['pixel_values', 'pixel_mask', 'labels']
for chave_original in chaves_originais:
    dados_da_tabela.pop(chave_original)

# Agora o dicionário 'dados_da_tabela' contém informações completas de cada célula detectada
print(dados_da_tabela)


In [ ]:
# Verificar o tipo de dados do tensor 'pixel_values'
print(type(dados_da_tabela['pixel_values']))

# Verificar a forma do tensor 'pixel_values'
print(dados_da_tabela['pixel_values'].shape)

# Verificar o tipo de dados do tensor 'labels'
print(type(dados_da_tabela['labels']))

# Verificar a forma do tensor 'labels'
print(dados_da_tabela['labels'].shape)




In [ ]:
# Verificar os elementos do tensor 'pixel_values'
print(dados_da_tabela['pixel_values'])

# Verificar os elementos do tensor 'labels'
print(dados_da_tabela['labels'])


In [ ]:
# Remover as chaves originais, pois foram substituídas pelas chaves com informações completas
chaves_originais = list(range(len(dados_da_tabela)))
for chave_original in chaves_originais:
    dados_da_tabela.pop(chave_original)

In [ ]:
celulas_detectadas = []
for i in range(len(dados_da_tabela)):
    retangulo = dados_da_tabela[i]
    img_celula = recortar_imagem(file_path, retangulo)
    texto_celula = pytesseract.image_to_string(img_celula, lang='por')
    
    # Mapear o ID numérico do rótulo para o rótulo descritivo usando model.config.id2label
    label_id = labels_ids[i].item() if isinstance(labels_ids[i], torch.Tensor) else labels_ids[i]
    label_descritivo = model.config.id2label[label_id]
    
    # Criar objeto CelulaTabela e adicioná-lo à lista de células_detectadas
    celula = CelulaTabela(coordenadas=retangulo, label=label_descritivo, texto=texto_celula)
    celulas_detectadas.append(celula)

# Agora a lista celulas_detectadas contém objetos CelulaTabela com informações de cada célula detectada
# Você pode acessar as informações de cada célula usando os atributos da classe
for celula in celulas_detectadas:
    print("Coordenadas:", celula.coordenadas)
    print("Label:", celula.label)
    print("Texto:", celula.texto)
    print("-" * 30)

In [ ]:
# Função para recortar a imagem com base nas coordenadas do retângulo
def recortar_imagem(img, coordenadas):
    x_min, y_min, x_max, y_max = coordenadas
    return img[int(y_min):int(y_max), int(x_min):int(x_max)]

# Processo de OCR para cada célula e preenchimento do dicionário
for i, retangulo in enumerate(retangulos):
    img_celula = recortar_imagem(img_pdf, retangulo)
    texto_celula = pytesseract.image_to_string(img_celula, lang='por')
    
    # Mapear o ID numérico do rótulo para o rótulo descritivo usando model.config.id2label
    label_id = labels_ids[i]
    label_descritivo = model.config.id2label[label_id]
    
    # Adicionar as informações ao dicionário
    chave_celula = f'celula_{i}'
    dados_da_tabela[chave_celula] = {
        'coordenadas': retangulo,
        'label': label_descritivo,
        'texto': texto_celula
    }

# Agora o dicionário 'dados_da_tabela' contém informações completas de cada célula detectada
print(dados_da_tabela)


In [ ]:

dados_da_tabela = {}
for i, retangulo in enumerate(retangulos):
    img_celula = recortar_imagem(img_pdf, retangulo)
    texto_celula = pytesseract.image_to_string(img_celula, lang='por')
    chave_celula = f'celula_{i}'  # Ou use outra forma de identificar a célula, se preferir
    dados_da_tabela[chave_celula] = texto_celula

# Agora 'dados_da_tabela' é um dicionário que mapeia cada célula ao seu conteúdo lido pelo OCR.
print(dados_da_tabela)

In [ ]:
retangulos

In [ ]:
labels_ids

In [ ]:
for dado in dados_da_tabela.items():
    print(dado)

In [ ]:

# Exemplo de resultados de OCR para cada célula
texto_das_celulas = ["Conteúdo da célula 1", "Conteúdo da célula 2", "..."]

# Criação do dicionário para armazenar os dados de cada detecção
dados_da_tabela = []

for i, label_id in enumerate(labels_ids):
    # Obtém o rótulo descritivo com base no ID numérico usando model.config.id2label
    rótulo_descritivo = model.config.id2label[label_id]

    # Obtém o texto correspondente à célula detectada usando o OCR
    texto_celula = texto_das_celulas[i]

    # Armazena todas as informações relevantes sobre a detecção em um dicionário
    detecao = {
        'label': rótulo_descritivo,
        'texto': texto_celula,
        # Adicione aqui outras informações relevantes, como coordenadas do retângulo, confiança da detecção, etc.
    }

    # Adiciona o dicionário da detecção à lista geral de dados da tabela
    dados_da_tabela.append(detecao)

# Agora 'dados_da_tabela' é uma lista de dicionários, cada dicionário contendo informações sobre uma detecção.
print(dados_da_tabela)


### Using Thinker